# Projecting Food Insecurity Rates in the US by County
## Data Collection and Preliminary Cleaning
The following process imports and cleans 7 different datasets, each broken down into a yearly dataset, spanning years 2009-2020.
### Flatiron School Data Science Capstone<br>By Khyatee Desai

In [30]:
# import necessary libraries
import pandas as pd
import numpy as np
import os
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# 1. Food Insecurity Data
This dataset contains data on food insecurity rates in the US by county, from 2009-2018.<br>**Source:** Feeding America [Map the Meal Gap Study](https://www.feedingamerica.org/research/map-the-meal-gap/how-we-got-the-map-data)
### Import all the excel files

In [31]:
directory = "../datasets/feeding_america/"

df_FA_09 = pd.read_excel(directory+'FA_2011_2009.xlsx')
df_FA_10 = pd.read_excel(directory+'FA_2012_2010.xlsx')
df_FA_11 = pd.read_excel(directory+'FA_2013_2011.xlsx') # data only provided at the state level
df_FA_12 = pd.read_excel(directory+'FA_2014_2012.xlsx') # data only provided at the state level
df_FA_13 = pd.read_excel(directory+'FA_2015_2013.xlsx') ## data only provided at the state level
df_FA_14 = pd.read_excel(directory+'FA_2016_2014.xlsx')
df_FA_15 = pd.read_excel(directory+'FA_2017_2015.xlsx')
df_FA_16 = pd.read_excel(directory+'FA_2018_2016.xlsx')
df_FA_17 = pd.read_excel(directory+'FA_2019_2017.xlsx')
df_FA_18 = pd.read_excel(directory+'FA_2020_2018.xlsx',header=1)

# projected data for 2020 - ignoring this for project MVP
# df_FAprojection_20 = pd.read_excel(directory+'projection_10.2020.xlsx')


In [32]:
# inspect the 2010 data
df_FA_10

,FIPS,State,"County, State",2010 Food Insecurity Rate,Number of Food Insecure Persons in 2010,Low Threshold in state,Low Threshold Type,High Threshold in state,High Threshold Type,% FI ≤ Low Threshold,% FI Btwn Thresholds,% FI > High Threshold,2010 Child food insecurity rate,Number of Food Insecure Children in 2010,% food insecure children in HH w/ HH incomes below 185 FPL,% of food insecure children in HH w/ HH incomes above 185 FPL,2010 Cost Per Meal,2010 Weighted Annual Food Budget Shortfall
0,1001,AL,"Autauga County, Alabama",0.13,7140,1.3,SNAP,1.85,Other Nutrition Program,0.33,0.21,0.47,0.20,2980.0,0.51,0.49,2.58,3170830
1,1003,AL,"Baldwin County, Alabama",0.13,23570,1.3,SNAP,1.85,Other Nutrition Program,0.35,0.29,0.37,0.24,9720.0,0.59,0.41,2.64,10710730
2,1005,AL,"Barbour County, Alabama",0.23,6440,1.3,SNAP,1.85,Other Nutrition Program,0.48,0.17,0.35,0.26,1600.0,0.87,0.13,2.53,2804540
3,1007,AL,"Bibb County, Alabama",0.16,3550,1.3,SNAP,1.85,Other Nutrition Program,0.36,0.29,0.35,0.25,1300.0,0.64,0.36,2.55,1558200
4,1009,AL,"Blount County, Alabama",0.13,7160,1.3,SNAP,1.85,Other Nutrition Program,0.41,0.30,0.28,0.25,3540.0,0.53,0.47,2.50,3081120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56037,WY,"Sweetwater County, Wyoming",0.11,4720,1.3,SNAP,1.85,Other Nutrition Program,0.27,0.16,0.56,0.18,2040.0,0.40,0.60,2.54,2063630
3139,56039,WY,"Teton County, Wyoming",0.12,2540,1.3,SNAP,1.85,Other Nutrition Program,0.31,0.21,0.49,0.20,840.0,0.55,0.45,3.48,1521490
3140,56041,WY,"Uinta County, Wyoming",0.13,2620,1.3,SNAP,1.85,Other Nutrition Program,0.37,0.19,0.44,0.19,1160.0,0.48,0.52,2.38,1073330
3141,56043,WY,"Washakie County, Wyoming",0.11,900,1.3,SNAP,1.85,Other Nutrition Program,0.35,0.29,0.36,0.15,300.0,0.59,0.41,2.44,378000


### Drop unnecessary features for each year
These features are removed due to either being unnecessary for the model, or being highly correlated with the target variable, which would create data leakage.

In [33]:
df_FA_09 = df_FA_09.drop(['State Name', 'County Code','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
            '% FI Btwn Thresholds','% FI > High Threshold', '% of children in FI HH with HH incomes at or below 185% FPL',
              'Number Food Insecure Children','% of children in FI HH with HH incomes above 185% FPL'], axis=1)

In [34]:
df_FA_10 = df_FA_10.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
              '% FI Btwn Thresholds', 'Number of Food Insecure Children in 2010 ',
              '% FI > High Threshold', '% food insecure children in HH w/ HH incomes below 185 FPL',
               '% of food insecure children in HH w/ HH incomes above 185 FPL'], axis=1)

In [35]:
df_FA_11 = df_FA_11.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                          '% FI Btwn Thresholds', 'Number of Food Insecure Children in 2011',
              '% FI > High Threshold', '% food insecure children in HH w/ HH incomes below 185 FPL',
               '% of food insecure children in HH w/ HH incomes above 185 FPL'], axis=1)

In [36]:
df_FA_12 = df_FA_12.drop(['State Name', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds','% FI > High Threshold', '# of Food Insecure Children in 2012',
               '% food insecure Children in HH w/HH Incomes Below 185 FPL in 2012',
               '% food insecure Children in HH w/HH Incomes Above 185 FPL in 2012'], axis=1)

In [37]:
df_FA_13 = df_FA_13.drop(['State Name', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds','% FI > High Threshold', '# of Food Insecure Children in 2013',
               '% food insecure Children in HH w/HH Incomes Below 185 FPL in 2013',
               '% food insecure Children in HH w/HH Incomes Above 185 FPL in 2013'], axis=1)

In [38]:
df_FA_14 = df_FA_14.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds', '% FI > High Threshold', '# of Food Insecure Children in 2014',
               '% food insecure children in HH w/ HH incomes below 185 FPL in 2014',
               '% food insecure children in HH w/ HH incomes above 185 FPL in 2014'], axis=1)

In [39]:
df_FA_15 = df_FA_15.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds','% FI > High Threshold', '# of Food Insecure Children in 2015',
               '% food insecure children in HH w/ HH incomes below 185 FPL in 2015',
               '% food insecure children in HH w/ HH incomes above 185 FPL in 2015'], axis=1)

In [40]:
df_FA_16 = df_FA_16.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds','% FI > High Threshold', '# of Food Insecure Children in 2016',
               '% food insecure children in HH w/ HH incomes below 185 FPL in 2016',
               '% food insecure children in HH w/ HH incomes above 185 FPL in 2016'], axis=1)

In [41]:
df_FA_17 = df_FA_17.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds', '% FI > High Threshold', '# of Food Insecure Children in 2017',
               '% food insecure children in HH w/ HH incomes below 185 FPL in 2017',
               '% food insecure children in HH w/ HH incomes above 185 FPL in 2017'], axis=1)

In [42]:
df_FA_18 = df_FA_18.drop(['County, State', 'State','Low Threshold in state', 'High Threshold in state', '% FI ≤ Low Threshold',
                '% FI Btwn Thresholds', '% FI > High Threshold', '# of Food Insecure Children in 2018',
               '% food insecure children in HH w/ HH incomes below 185 FPL in 2018',
               '% food insecure children in HH w/ HH incomes above 185 FPL in 2018'], axis=1)

### Reformat FIPS Column
FIPS, also referred to as GEOID, is a unique code assigned to each County/State combination. FIPS sometimes contains a leading 0, which was ommitted during csv to pandas conversion, so the following process is used to re-add the leading zeroes.

In [43]:
# 2009
# drop null rows at the end
df_FA_09.drop(df_FA_09[df_FA_09['FIPS'].isnull()].index, axis=0, inplace=True)
# change FIPS to string and add leading zeros if needed
df_FA_09['FIPS'] = np.where(df_FA_09['FIPS']<10000, 
                        '0'+df_FA_09['FIPS'].astype(int).astype(str), df_FA_09['FIPS'].astype(int).astype(str))
# 2010
df_FA_10.drop(df_FA_10[df_FA_10['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_10['FIPS'] = np.where(df_FA_10['FIPS']<10000, 
                        '0'+df_FA_10['FIPS'].astype(int).astype(str), df_FA_10['FIPS'].astype(int).astype(str))
# 2011
df_FA_11.drop(df_FA_11[df_FA_11['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_11['FIPS'] = np.where(df_FA_11['FIPS']<10000, 
                        '0'+df_FA_11['FIPS'].astype(int).astype(str), df_FA_11['FIPS'].astype(int).astype(str))
# 2012
df_FA_12.drop(df_FA_12[df_FA_12['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_12['FIPS'] = np.where(df_FA_12['FIPS']<10000, 
                        '0'+df_FA_12['FIPS'].astype(int).astype(str), df_FA_12['FIPS'].astype(int).astype(str))
# 2013
df_FA_13.drop(df_FA_13[df_FA_13['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_13['FIPS'] = np.where(df_FA_13['FIPS']<10000, 
                        '0'+df_FA_13['FIPS'].astype(int).astype(str), df_FA_13['FIPS'].astype(int).astype(str))
# 2014
df_FA_14.drop(df_FA_14[df_FA_14['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_14['FIPS'] = np.where(df_FA_14['FIPS']<10000, 
                        '0'+df_FA_14['FIPS'].astype(int).astype(str), df_FA_14['FIPS'].astype(int).astype(str))
# 2015
df_FA_15.drop(df_FA_15[df_FA_15['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_15['FIPS'] = np.where(df_FA_15['FIPS']<10000, 
                        '0'+df_FA_15['FIPS'].astype(int).astype(str), df_FA_15['FIPS'].astype(int).astype(str))
# 2016
df_FA_16.drop(df_FA_16[df_FA_16['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_16['FIPS'] = np.where(df_FA_16['FIPS']<10000, 
                        '0'+df_FA_16['FIPS'].astype(int).astype(str), df_FA_16['FIPS'].astype(int).astype(str))
# 2017
df_FA_17.drop(df_FA_17[df_FA_17['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_17['FIPS'] = np.where(df_FA_17['FIPS']<10000, 
                        '0'+df_FA_17['FIPS'].astype(int).astype(str), df_FA_17['FIPS'].astype(int).astype(str))
# 2018
df_FA_18.drop(df_FA_18[df_FA_18['FIPS'].isnull()].index, axis=0, inplace=True)
df_FA_18['FIPS'] = np.where(df_FA_18['FIPS']<10000, 
                        '0'+df_FA_18['FIPS'].astype(int).astype(str), df_FA_18['FIPS'].astype(int).astype(str))



### Create Year column for each df
This will be relevant further on, when the datasets are vertically concatenated together.

In [44]:
df_FA_09['Year'] = '2009'
df_FA_10['Year'] = '2010'
df_FA_11['Year'] = '2011'
df_FA_12['Year'] = '2012'
df_FA_13['Year'] = '2013'
df_FA_14['Year'] = '2014'
df_FA_15['Year'] = '2015'
df_FA_16['Year'] = '2016'
df_FA_17['Year'] = '2017'
df_FA_18['Year'] = '2018'

### Rename columns for uniformity

In [45]:
df_FA_10.rename(columns={'2010 Food Insecurity Rate':'FI Rate', 'Number of Food Insecure Persons in 2010':'Number Food Insecure Individuals',
       '2010 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2010 Cost Per Meal':'Cost Per Meal', 
                         '2010 Child food insecurity rate':'Child FI Rate'}, inplace=True)
df_FA_11.rename(columns={'2011 Food Insecurity Rate':'FI Rate', 'Number of Food Insecure Persons in 2011':'Number Food Insecure Individuals',
       '2011 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2011 Cost Per Meal':'Cost Per Meal', 
                         '2011 Child Food Insecurity Rate':'Child FI Rate'}, inplace=True)
df_FA_12.rename(columns={'2012 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2012 ':'Number Food Insecure Individuals',
       '2012 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2012 Cost Per Meal':'Cost Per Meal', 
                         '2012 Child Food Insecurity Rate':'Child FI Rate'}, inplace=True)
df_FA_13.rename(columns={'2013 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2013 ':'Number Food Insecure Individuals',
       '2013 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2013 Cost Per Meal':'Cost Per Meal', 
                         '2013 Child Food Insecurity Rate':'Child FI Rate'}, inplace=True)
df_FA_14.rename(columns={'2014 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2014':'Number Food Insecure Individuals',
       '2014 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2014 Cost Per Meal':'Cost Per Meal', 
                         '2014 Child food insecurity rate':'Child FI Rate'}, inplace=True)
df_FA_15.rename(columns={'2015 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2015':'Number Food Insecure Individuals',
       '2015 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2015 Cost Per Meal':'Cost Per Meal', 
                         '2015 Child food insecurity rate':'Child FI Rate'}, inplace=True)
df_FA_16.rename(columns={'2016 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2016':'Number Food Insecure Individuals',
       '2016 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2016 Cost Per Meal':'Cost Per Meal', 
                         '2016 Child food insecurity rate':'Child FI Rate'}, inplace=True)
df_FA_17.rename(columns={'2017 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2017':'Number Food Insecure Individuals',
       '2017 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2017 Cost Per Meal':'Cost Per Meal', 
                         '2017 Child food insecurity rate':'Child FI Rate'}, inplace=True)
df_FA_18.rename(columns={'2018 Food Insecurity Rate':'FI Rate', '# of Food Insecure Persons in 2018':'Number Food Insecure Individuals',
       '2018 Weighted Annual Food Budget Shortfall':'Weighted Annual Dollars', '2018 Cost Per Meal':'Cost Per Meal', 
                         '2018 Child food insecurity rate':'Child FI Rate'}, inplace=True)

### Concatenate all df's to create master dataframe for all years
df_FA is the cleaned Food Insecurity data, from all years of the study.

In [191]:
df_FA = pd.concat([df_FA_09, df_FA_10,df_FA_11,df_FA_12,df_FA_13,df_FA_14,df_FA_15,df_FA_16,df_FA_17,
    df_FA_18 ]).drop(['Weighted Annual Dollars', 'Number Food Insecure Individuals','Child FI Rate'],axis=1)
df_FA

,FIPS,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Year
0,02013,0.15,SNAP,other nutrition pgm,NaN,2009
1,02016,0.14,SNAP,other nutrition pgm,NaN,2009
2,02020,0.12,SNAP,other nutrition pgm,NaN,2009
3,02050,0.21,SNAP,other nutrition pgm,NaN,2009
4,02060,0.10,SNAP,other nutrition pgm,NaN,2009
...,...,...,...,...,...,...
3137,56037,0.12,SNAP,Other Nutrition Program,3.29,2018
3138,56039,0.10,SNAP,Other Nutrition Program,4.52,2018
3139,56041,0.14,SNAP,Other Nutrition Program,3.07,2018
3140,56043,0.13,SNAP,Other Nutrition Program,3.26,2018


# 2. Unemployment Data
This dataset contains yearly data on the Labor Force of each US county, for the years 2009-2019. The files include data on total workforce, and unemployment rates.<br>
**Source:** [Bureau of Labor Statistics](https://www.bls.gov/lau/#tables)
### Import the datasets for each year

In [47]:
directory = "../datasets/unemployment/"

df_unemp_09 = pd.read_excel(directory + 'laucnty09.xlsx', header=4).drop(0,axis=0)
df_unemp_10 = pd.read_excel(directory + 'laucnty10.xlsx', header=4).drop(0,axis=0)
df_unemp_11 = pd.read_excel(directory + 'laucnty11.xlsx', header=4).drop(0,axis=0)
df_unemp_12 = pd.read_excel(directory + 'laucnty12.xlsx', header=4).drop(0,axis=0)
df_unemp_13 = pd.read_excel(directory + 'laucnty13.xlsx', header=4).drop(0,axis=0)
df_unemp_14 = pd.read_excel(directory + 'laucnty14.xlsx', header=4).drop(0,axis=0)
df_unemp_15 = pd.read_excel(directory + 'laucnty15.xlsx', header=4).drop(0,axis=0)
df_unemp_16 = pd.read_excel(directory + 'laucnty16.xlsx', header=4).drop(0,axis=0)
df_unemp_17 = pd.read_excel(directory + 'laucnty17.xlsx', header=4).drop(0,axis=0)
df_unemp_18 = pd.read_excel(directory + 'laucnty18.xlsx', header=4).drop(0,axis=0)
df_unemp_19 = pd.read_excel(directory + 'laucnty19.xlsx', header=4).drop(0,axis=0)


In [48]:
# Inspect one of the files
df_unemp_10.head(3)

,Code,Code.1,Code.2,County Name/State Abbreviation,Year,Unnamed: 5,Force,Employed,Unemployed,(%)
1,CN0100100000000,1.0,1.0,"Autauga County, AL",2010.0,NaN,25713.0,23431.0,2282.0,8.9
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",2010.0,NaN,83459.0,75120.0,8339.0,10.0
3,CN0100500000000,1.0,5.0,"Barbour County, AL",2010.0,NaN,10221.0,8959.0,1262.0,12.3


### Rename columns for interpretability

In [49]:
df_unemp_09.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [50]:
df_unemp_10.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [51]:
df_unemp_11.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [52]:
df_unemp_12.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [53]:
df_unemp_13.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [54]:
df_unemp_14.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [55]:
df_unemp_15.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [56]:
df_unemp_16.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [57]:
df_unemp_17.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [58]:
df_unemp_18.rename(columns = {'Code':'CN', 'Code.1':'FIPS_state', 'Code.2':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 'Unnamed: 5': 'idk',
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

In [59]:
df_unemp_19.rename(columns = {'LAUS Code':'CN', 'Code':'FIPS_state', 'Code.1':'FIPS_county', 
                             'County Name/State Abbreviation': 'State/County', 
                              'Force':'Total_workforce','(%)':'Unemployment_rate'}, inplace=True)

### Drop null rows, drop some columns, and reformat year column
Drop null rows (created during excel conversion,) drop columns that wont be used during modeling, and reformat the year column as a string, to match the other datasets.

In [60]:
# 2009
# drop last three rows which were null
df_unemp_09.drop(df_unemp_09[df_unemp_09['FIPS_state'].isnull()].index, inplace=True)
# change year column to string
df_unemp_09['Year'] = df_unemp_09['Year'].astype(int).astype(str)
# drop unneeded columns
df_unemp_09.drop(['CN', 'idk'], axis=1, inplace=True)

# 2010
df_unemp_10.drop(df_unemp_10[df_unemp_10['FIPS_state'].isnull()].index, inplace=True)
df_unemp_10['Year'] = df_unemp_10['Year'].astype(int).astype(str)
df_unemp_10.drop(['CN', 'idk'], axis=1, inplace=True)

# 2011
df_unemp_11.drop(df_unemp_11[df_unemp_11['FIPS_state'].isnull()].index, inplace=True)
df_unemp_11['Year'] = df_unemp_11['Year'].astype(int).astype(str)
df_unemp_11.drop(['CN', 'idk'], axis=1, inplace=True)

# 2012
df_unemp_12.drop(df_unemp_12[df_unemp_12['FIPS_state'].isnull()].index, inplace=True)
df_unemp_12['Year'] = df_unemp_12['Year'].astype(int).astype(str)
df_unemp_12.drop(['CN', 'idk'], axis=1, inplace=True)

# 2013
df_unemp_13.drop(df_unemp_13[df_unemp_13['FIPS_state'].isnull()].index, inplace=True)
df_unemp_13['Year'] = df_unemp_13['Year'].astype(int).astype(str)
df_unemp_13.drop(['CN', 'idk'], axis=1, inplace=True)

# 2014
df_unemp_14.drop(df_unemp_14[df_unemp_14['FIPS_state'].isnull()].index, inplace=True)
df_unemp_14['Year'] = df_unemp_14['Year'].astype(int).astype(str)
df_unemp_14.drop(['CN', 'idk'], axis=1, inplace=True)

# 2015
df_unemp_15.drop(df_unemp_15[df_unemp_15['FIPS_state'].isnull()].index, inplace=True)
df_unemp_15['Year'] = df_unemp_15['Year'].astype(int).astype(str)
df_unemp_15.drop(['CN', 'idk'], axis=1, inplace=True)

# 2016
df_unemp_16.drop(df_unemp_16[df_unemp_16['FIPS_state'].isnull()].index, inplace=True)
df_unemp_16['Year'] = df_unemp_16['Year'].astype(int).astype(str)
df_unemp_16.drop(['CN', 'idk'], axis=1, inplace=True)

# 2017
df_unemp_17.drop(df_unemp_17[df_unemp_17['FIPS_state'].isnull()].index, inplace=True)
df_unemp_17['Year'] = df_unemp_17['Year'].astype(int).astype(str)
df_unemp_17.drop(['CN', 'idk'], axis=1, inplace=True)

# 2018
df_unemp_18.drop(df_unemp_18[df_unemp_18['FIPS_state'].isnull()].index, inplace=True)
df_unemp_18['Year'] = df_unemp_18['Year'].astype(int).astype(str)
df_unemp_18.drop(['CN', 'idk'], axis=1, inplace=True)

# 2019
# drop last three rows which were null
df_unemp_19.drop(df_unemp_19[df_unemp_19['FIPS_state'].isnull()].index, inplace=True)
df_unemp_19.drop(['CN'], axis=1, inplace=True)


### Reformat FIPS
Complete FIPS are a combination of a State FIPS + County FIPS code. The following process creates the complete code for each observation, by adding together the values from both columns. Similarly to above, a leading zero is also added in where needed, as it was dropped during the CSV to Pandas conversion.

In [61]:
# 2009
# add leading zeros to FIPS codes and convert to string
df_unemp_09['FIPS_county'] = np.select([df_unemp_09['FIPS_county']<10, df_unemp_09['FIPS_county']<100],
                    ['00'+df_unemp_09['FIPS_county'].astype(int).astype(str), '0'+df_unemp_09['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_09['FIPS_county'].astype(int).astype(str))
df_unemp_09['FIPS_state'] = np.where(df_unemp_09['FIPS_state']<10, 
                        '0'+df_unemp_09['FIPS_state'].astype(int).astype(str), df_unemp_09['FIPS_state'].astype(int).astype(str))
# Create main fips code
df_unemp_09['FIPS'] = df_unemp_09['FIPS_state'] + df_unemp_09['FIPS_county']

# 2010
df_unemp_10['FIPS_county'] = np.select([df_unemp_10['FIPS_county']<10, df_unemp_10['FIPS_county']<100],
                    ['00'+df_unemp_10['FIPS_county'].astype(int).astype(str), '0'+df_unemp_10['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_10['FIPS_county'].astype(int).astype(str))
df_unemp_10['FIPS_state'] = np.where(df_unemp_10['FIPS_state']<10, 
                        '0'+df_unemp_10['FIPS_state'].astype(int).astype(str), df_unemp_10['FIPS_state'].astype(int).astype(str))
df_unemp_10['FIPS'] = df_unemp_10['FIPS_state'] + df_unemp_10['FIPS_county']

# 2011
df_unemp_11['FIPS_county'] = np.select([df_unemp_11['FIPS_county']<10, df_unemp_11['FIPS_county']<100],
                    ['00'+df_unemp_11['FIPS_county'].astype(int).astype(str), '0'+df_unemp_11['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_11['FIPS_county'].astype(int).astype(str))
df_unemp_11['FIPS_state'] = np.where(df_unemp_11['FIPS_state']<10, 
                        '0'+df_unemp_11['FIPS_state'].astype(int).astype(str), df_unemp_11['FIPS_state'].astype(int).astype(str))
df_unemp_11['FIPS'] = df_unemp_11['FIPS_state'] + df_unemp_11['FIPS_county']

# 2012
df_unemp_12['FIPS_county'] = np.select([df_unemp_12['FIPS_county']<10, df_unemp_12['FIPS_county']<100],
                    ['00'+df_unemp_12['FIPS_county'].astype(int).astype(str), '0'+df_unemp_12['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_12['FIPS_county'].astype(int).astype(str))
df_unemp_12['FIPS_state'] = np.where(df_unemp_12['FIPS_state']<10, 
                        '0'+df_unemp_12['FIPS_state'].astype(int).astype(str), df_unemp_12['FIPS_state'].astype(int).astype(str))
df_unemp_12['FIPS'] = df_unemp_12['FIPS_state'] + df_unemp_12['FIPS_county']

# 2013
df_unemp_13['FIPS_county'] = np.select([df_unemp_13['FIPS_county']<10, df_unemp_13['FIPS_county']<100],
                    ['00'+df_unemp_13['FIPS_county'].astype(int).astype(str), '0'+df_unemp_13['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_13['FIPS_county'].astype(int).astype(str))
df_unemp_13['FIPS_state'] = np.where(df_unemp_13['FIPS_state']<10, 
                        '0'+df_unemp_13['FIPS_state'].astype(int).astype(str), df_unemp_13['FIPS_state'].astype(int).astype(str))
df_unemp_13['FIPS'] = df_unemp_13['FIPS_state'] + df_unemp_13['FIPS_county']

# 2014
df_unemp_14['FIPS_county'] = np.select([df_unemp_14['FIPS_county']<10, df_unemp_14['FIPS_county']<100],
                    ['00'+df_unemp_14['FIPS_county'].astype(int).astype(str), '0'+df_unemp_14['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_14['FIPS_county'].astype(int).astype(str))
df_unemp_14['FIPS_state'] = np.where(df_unemp_14['FIPS_state']<10, 
                        '0'+df_unemp_14['FIPS_state'].astype(int).astype(str), df_unemp_14['FIPS_state'].astype(int).astype(str))
df_unemp_14['FIPS'] = df_unemp_14['FIPS_state'] + df_unemp_14['FIPS_county']

# 2015
df_unemp_15['FIPS_county'] = np.select([df_unemp_15['FIPS_county']<10, df_unemp_15['FIPS_county']<100],
                    ['00'+df_unemp_15['FIPS_county'].astype(int).astype(str), '0'+df_unemp_15['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_15['FIPS_county'].astype(int).astype(str))
df_unemp_15['FIPS_state'] = np.where(df_unemp_15['FIPS_state']<10, 
                        '0'+df_unemp_15['FIPS_state'].astype(int).astype(str), df_unemp_15['FIPS_state'].astype(int).astype(str))
df_unemp_15['FIPS'] = df_unemp_15['FIPS_state'] + df_unemp_15['FIPS_county']

# 2016
df_unemp_16['FIPS_county'] = np.select([df_unemp_16['FIPS_county']<10, df_unemp_16['FIPS_county']<100],
                    ['00'+df_unemp_16['FIPS_county'].astype(int).astype(str), '0'+df_unemp_16['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_16['FIPS_county'].astype(int).astype(str))
df_unemp_16['FIPS_state'] = np.where(df_unemp_16['FIPS_state']<10, 
                        '0'+df_unemp_16['FIPS_state'].astype(int).astype(str), df_unemp_16['FIPS_state'].astype(int).astype(str))
df_unemp_16['FIPS'] = df_unemp_16['FIPS_state'] + df_unemp_16['FIPS_county']

# 2017
df_unemp_17['FIPS_county'] = np.select([df_unemp_17['FIPS_county']<10, df_unemp_17['FIPS_county']<100],
                    ['00'+df_unemp_17['FIPS_county'].astype(int).astype(str), '0'+df_unemp_17['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_17['FIPS_county'].astype(int).astype(str))
df_unemp_17['FIPS_state'] = np.where(df_unemp_17['FIPS_state']<10, 
                        '0'+df_unemp_17['FIPS_state'].astype(int).astype(str), df_unemp_17['FIPS_state'].astype(int).astype(str))
df_unemp_17['FIPS'] = df_unemp_17['FIPS_state'] + df_unemp_17['FIPS_county']

# 2018
df_unemp_18['FIPS_county'] = np.select([df_unemp_18['FIPS_county']<10, df_unemp_18['FIPS_county']<100],
                    ['00'+df_unemp_18['FIPS_county'].astype(int).astype(str), '0'+df_unemp_18['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_18['FIPS_county'].astype(int).astype(str))
df_unemp_18['FIPS_state'] = np.where(df_unemp_18['FIPS_state']<10, 
                        '0'+df_unemp_18['FIPS_state'].astype(int).astype(str), df_unemp_18['FIPS_state'].astype(int).astype(str))
df_unemp_18['FIPS'] = df_unemp_18['FIPS_state'] + df_unemp_18['FIPS_county']

# 2019
df_unemp_19['FIPS_county'] = np.select([df_unemp_19['FIPS_county']<10, df_unemp_19['FIPS_county']<100],
                    ['00'+df_unemp_19['FIPS_county'].astype(int).astype(str), '0'+df_unemp_19['FIPS_county'].astype(int).astype(str)],
                    default= df_unemp_19['FIPS_county'].astype(int).astype(str))
df_unemp_19['FIPS_state'] = np.where(df_unemp_19['FIPS_state']<10, 
                        '0'+df_unemp_19['FIPS_state'].astype(int).astype(str), df_unemp_19['FIPS_state'].astype(int).astype(str))
df_unemp_19['FIPS'] = df_unemp_19['FIPS_state'] + df_unemp_19['FIPS_county']


### Break down 2019 dataset into 2019 and 2020 dataframes
The file for 2019 contains data on both 2019 and 2020, so this is broken down into separate dataframes.

In [62]:
# Add year column for each, derived from Period column, and then drop period column
df_unemp_20 = df_unemp_19[df_unemp_19['Period'].str.contains('20')]
df_unemp_20['Year'] = '2020'
df_unemp_20.drop('Period', axis=1, inplace=True)

df_unemp_19 = df_unemp_19[df_unemp_19['Period'].str.contains('19')]
df_unemp_19['Year'] = '2019'
df_unemp_19.drop('Period', axis=1, inplace=True)

### Concatenate all df's to create master dataframe of all years
df_unemployment is the final cleaned dataframe on unemployment data from all years.

In [63]:
df_unemployment = pd.concat([df_unemp_09, df_unemp_10,df_unemp_11,df_unemp_12,df_unemp_13,df_unemp_14,df_unemp_15,df_unemp_16,
          df_unemp_17,df_unemp_18,df_unemp_19,df_unemp_20])
df_unemployment.head()


,FIPS_state,FIPS_county,State/County,Year,Total_workforce,Employed,Unemployed,Unemployment_rate,FIPS
1,01,001,"Autauga County, AL",2009,2.5e+04,2.2e+04,2.4e+03,9.7,01001
2,01,003,"Baldwin County, AL",2009,8.2e+04,7.4e+04,8e+03,9.8,01003
3,01,005,"Barbour County, AL",2009,1e+04,8.6e+03,1.4e+03,14,01005
4,01,007,"Bibb County, AL",2009,8.7e+03,7.6e+03,1.2e+03,13,01007
5,01,009,"Blount County, AL",2009,2.6e+04,2.4e+04,2.6e+03,10,01009
...,...,...,...,...,...,...,...,...,...
45062,72,145,"Vega Baja Municipio, PR",2020,12543,11146,1397,11,72145
45063,72,147,"Vieques Municipio, PR",2020,2386,2133,253,11,72147
45064,72,149,"Villalba Municipio, PR",2020,6603,5969,634,9.6,72149
45065,72,151,"Yabucoa Municipio, PR",2020,7961,7168,793,10,72151


# 3a. Current Population Survey Data
The CPS survey is an annual survey run by the Census Bureau to gather information on household income, demographics, and other attributes. Currently, the data has only been collected for 2019 and 2020, ***therefore this data will not be used in the MVP modeling process.*<br>
[Data Dictionary 2019](https://www2.census.gov/programs-surveys/cps/techdocs/cpsmar19.pdf)<br>
[Data Dictionary 2020](https://www2.census.gov/programs-surveys/cps/datasets/2020/march/ASEC2020ddl_pub_full.pdf)<br>

**Source:** United States Census Bureau [Current Population Survey](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.html)

In [468]:
# Import the datasets
# df_household_19 = pd.read_csv('../datasets/household/hhpub19.csv')
df_household_20 = pd.read_csv('../datasets/household/hhpub20.csv')

### Map column values to data dictionary
A data dictionary is provided to map survey response codes to their meanings. The following cells map these codes using np.select:

In [470]:
# use np.select to map values on 2020 data
conditions=[df_household_20['GTMETSTA'] ==1,df_household_20['GTMETSTA'] ==2, df_household_20['GTMETSTA'] ==3]
choices = ['HH_Metrop', 'HH_Non-Metrop','N/A']
df_household_20['GTMETSTA'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['H_TENURE'] ==0,df_household_20['H_TENURE'] ==1,df_household_20['H_TENURE'] ==2, df_household_20['H_TENURE'] ==3]
choices = ['N/A', 'HH_owned', 'HH_rented','HH_rented_noCash']
df_household_20['H_TENURE'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['HDIS_YN'] ==0,df_household_20['HDIS_YN'] ==1,df_household_20['HDIS_YN'] ==2]
choices = ['N/A',  'HH_disabled','HH_not_disabled' ]
df_household_20['HDIS_YN'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['HCSP_YN'] ==0,df_household_20['HCSP_YN'] ==1,df_household_20['HCSP_YN'] ==2]
choices = ['N/A','HH_Child_support', 'HH_no_child_support' ]
df_household_20['HCSP_YN'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['HINC_UC'] ==0,df_household_20['HINC_UC'] ==1,df_household_20['HINC_UC'] ==2]
choices = ['N/A','HH_unemployment_pay', 'HH_no_unemployment_pay' ]
df_household_20['HINC_UC'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['NOW_HCOV'] ==1,df_household_20['NOW_HCOV'] ==2,df_household_20['NOW_HCOV'] ==3]
choices = [ 'HH_health_insured','HH_some_health_insured','HH_no_health_insured' ]
df_household_20['NOW_HCOV'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['NOW_HPUB'] ==1,df_household_20['NOW_HPUB'] ==2,df_household_20['NOW_HPUB'] ==3]
choices = [ 'HH_health_pub','HH_some_health_pub','HH_no_health_pub' ]
df_household_20['NOW_HPUB'] = np.select(conditions, choices,default='N/A')

conditions=[df_household_20['NOW_HPRIV'] ==1,df_household_20['NOW_HPRIV'] ==2,df_household_20['NOW_HPRIV'] ==3]
choices = [ 'HH_health_priv','HH_some_health_priv','HH_no_health_priv' ]
df_household_20['NOW_HPRIV'] = np.select(conditions, choices,default='N/A')


### Rename columns for interpretability

In [472]:
# rename 2020 data
df_household_20 = df_household_20.loc[:,['GESTFIPS', 'GTCO', 'GTMETSTA', 'HTOTVAL','H_NUMPER', 'HUNDER18',
                 'H_TENURE','HDIS_YN', 'HCSP_YN', 'HINC_UC','NOW_HCOV','NOW_HPUB','NOW_HPRIV']]
df_household_20 = df_household_20.rename(columns={'GESTFIPS':'FIPS_state', 'GTCO':'FIPS_county', 'GTMETSTA':'Metro_status',
                                'HTOTVAL':'HH_income',
                                'H_NUMPER':'HH_size', 'HUNDER18':'Num_minors','H_TENURE':'Rent_vs_Owned',
                               'HDIS_YN':'Disability', 'HCSP_YN':'Child_support', 'HINC_UC':'Unemployment_payments',
                               'NOW_HCOV':'Health_insurance', 'NOW_HPUB': 'Public_insurance', 'NOW_HPRIV':'Priv_insurance'})
df_household_20.head()

,FIPS_state,FIPS_county,Metro_status,HH_income,HH_size,Num_minors,Rent_vs_Owned,Disability,Child_support,Unemployment_payments,Health_insurance,Public_insurance,Priv_insurance
0,23,0,HH_Non-Metrop,127449,2,0,HH_owned,HH_not_disabled,HH_no_child_support,HH_no_unemployment_pay,HH_health_insured,HH_some_health_pub,HH_health_priv
1,23,0,HH_Non-Metrop,64680,2,0,HH_owned,HH_not_disabled,HH_no_child_support,HH_no_unemployment_pay,HH_health_insured,HH_some_health_pub,HH_health_priv
2,23,0,HH_Non-Metrop,40002,1,0,HH_owned,HH_not_disabled,HH_no_child_support,HH_no_unemployment_pay,HH_health_insured,HH_no_health_pub,HH_health_priv
3,23,0,HH_Non-Metrop,8424,2,0,HH_rented,HH_not_disabled,HH_no_child_support,HH_no_unemployment_pay,HH_health_insured,HH_health_pub,HH_no_health_priv
4,23,0,HH_Non-Metrop,59114,4,0,HH_owned,HH_not_disabled,HH_no_child_support,HH_no_unemployment_pay,HH_health_insured,HH_no_health_pub,HH_health_priv


### Create FIPS column from State FIPS and County FIPS

In [473]:
df_household_20['FIPS_county'] = np.select([df_household_20['FIPS_county']<10, df_household_20['FIPS_county']<100],
        ['00'+df_household_20['FIPS_county'].astype(int).astype(str), '0'+df_household_20['FIPS_county'].astype(int).astype(str)],
                    default= df_household_20['FIPS_county'].astype(int).astype(str))
df_household_20['FIPS_state'] = np.where(df_household_20['FIPS_state']<10, 
            '0'+df_household_20['FIPS_state'].astype(int).astype(str), df_household_20['FIPS_state'].astype(int).astype(str))
df_household_20['FIPS'] = df_household_20['FIPS_state'] + df_household_20['FIPS_county']


### Unstack Columns
Turn values into new columns for `Disability` and `Health_insurance` columns

In [474]:
# get population for each FIPS
pop_per_county = df_household_20['FIPS'].value_counts().reset_index().rename(columns={'FIPS':'Num_respondants_a','index':'FIPS'}).set_index('FIPS')


### Group by County Code (FIPS) and count the values for Disability and Health Insurance

In [475]:
# Unstack values into new columns for Disability column, group by FIPS and count
to_count = df_household_20.loc[:,['FIPS','Disability','Health_insurance','Public_insurance', 'Priv_insurance']]

not_disabled = to_count[to_count['Disability']=='HH_not_disabled'].groupby('FIPS')[['Disability' ]].count().rename(columns={'Disability':'HH_not_disabled'})
nan_disabled = to_count[to_count['Disability']=='N/A'].groupby('FIPS')[['Disability' ]].count().rename(columns={'Disability':'HH_nan_disabled'})
disabled = to_count[to_count['Disability']=='HH_disabled'].groupby('FIPS')[['Disability' ]].count().rename(columns={'Disability':'HH_disabled'})

df_disability = pd.concat([not_disabled,nan_disabled, disabled ],axis=1)


In [476]:
# Unstack values into new columns for Health Insurance column, group by FIPS and count
insured = to_count[to_count['Health_insurance']=='HH_health_insured'].groupby('FIPS')[['Health_insurance' ]].count().rename(columns={'Health_insurance':'HH_health_insured'})
nan_insured = to_count[to_count['Health_insurance']=='N/A'].groupby('FIPS')[['Health_insurance' ]].count().rename(columns={'Health_insurance':'HH_nan_insured'})
some_insured =to_count[to_count['Health_insurance']=='HH_some_health_insured'].groupby('FIPS')[['Health_insurance' ]].count().rename(columns={'Health_insurance':'HH_some_insured'})
not_insured =to_count[to_count['Health_insurance']=='HH_no_health_insured'].groupby('FIPS')[['Health_insurance' ]].count().rename(columns={'Health_insurance':'HH_not_insured'})

df_insurance = pd.concat([insured,nan_insured, some_insured,not_insured ],axis=1)


In [477]:
# Unstack values into new columns for Public Health Insurance column, group by FIPS and count
insured = to_count[to_count['Public_insurance']=='HH_no_health_pub'].groupby('FIPS')[['Public_insurance' ]].count().rename(columns={'Public_insurance':'HH_no_health_pub'})
nan_insured = to_count[to_count['Public_insurance']=='N/A'].groupby('FIPS')[['Public_insurance' ]].count().rename(columns={'Public_insurance':'HH_nan_insured_pub'})
some_insured =to_count[to_count['Public_insurance']=='HH_health_pub'].groupby('FIPS')[['Public_insurance' ]].count().rename(columns={'Public_insurance':'HH_insured_pub'})
not_insured =to_count[to_count['Public_insurance']=='HH_some_health_pub'].groupby('FIPS')[['Public_insurance' ]].count().rename(columns={'Public_insurance':'HH_some_insured_pub'})

df_insurance_pub = pd.concat([insured,nan_insured, some_insured,not_insured ],axis=1)


In [478]:
# Unstack values into new columns for Private Health Insurance column, group by FIPS and count
insured = to_count[to_count['Priv_insurance']=='HH_health_priv'].groupby('FIPS')[['Priv_insurance' ]].count().rename(columns={'Priv_insurance':'HH_health_priv'})
nan_insured = to_count[to_count['Priv_insurance']=='N/A'].groupby('FIPS')[['Priv_insurance' ]].count().rename(columns={'Priv_insurance':'HH_nan_insured_priv'})
some_insured =to_count[to_count['Priv_insurance']=='HH_no_health_priv'].groupby('FIPS')[['Priv_insurance' ]].count().rename(columns={'Priv_insurance':'HH_not_insured_priv'})
not_insured =to_count[to_count['Priv_insurance']=='HH_some_health_priv'].groupby('FIPS')[['Priv_insurance' ]].count().rename(columns={'Priv_insurance':'HH_some_insured_priv'})

df_insurance_priv = pd.concat([insured,nan_insured, some_insured,not_insured ],axis=1)

### Group by County Code (FIPS) and average the values for HH income and HH size

In [479]:
# Isolate columns that need to be grouped by FIPS and averaged
to_avg = df_household_20.loc[:,['FIPS','HH_income','HH_size']]
df_income_size = to_avg.groupby('FIPS')[['HH_income','HH_size']].mean()

### Concat new columns together

In [480]:
# Concat together the unstacked columns with value counts for each County
df_cps_20a = pd.concat([df_insurance, df_insurance_pub, df_insurance_priv, df_disability, df_income_size,pop_per_county], axis=1).fillna(0)

# # Rename FIPS column
# df_cps_20a.rename(columns={'index':'FIPS'},inplace=True)

# Add Year column
df_cps_20a['Year'] = '2020'
df_cps_20a.head()

,HH_health_insured,HH_nan_insured,HH_some_insured,HH_not_insured,HH_no_health_pub,HH_nan_insured_pub,HH_insured_pub,HH_some_insured_pub,HH_health_priv,HH_nan_insured_priv,HH_not_insured_priv,HH_some_insured_priv,HH_not_disabled,HH_nan_disabled,HH_disabled,HH_income,HH_size,Num_respondants_a,Year
01000,889,490,110.0,32.0,504,490,315.0,212.0,582,490,273.0,176.0,1002,490,29.0,53980.42,1.70,1521,2020
01003,86,58,3.0,2.0,36,58,42.0,13.0,51,58,29.0,11.0,88,58,3.0,52303.19,1.35,149,2020
01081,49,36,7.0,2.0,29,36,16.0,13.0,31,36,15.0,12.0,57,36,1.0,53911.45,1.61,94,2020
01097,97,44,11.0,7.0,71,44,22.0,22.0,76,44,24.0,15.0,113,44,2.0,50787.05,1.72,159,2020
02000,570,374,95.0,49.0,403,374,187.0,124.0,399,374,213.0,102.0,694,374,20.0,67311.89,1.92,1088,2020


# 3b. Basic CPS Data
The Basic CPS survey is a monthly survey of income, labor force participation, and demographic information, conducted by the U.S. Government and the US Census Bureau.
[Data Dictionary](https://www2.census.gov/programs-surveys/cps/datasets/2020/basic/2020_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt)<br>

**Source:** National Bureau of Economic Research [Current Population Survey - Basic Monthly](http://www2.nber.org/data/cps-basic2/)

In [481]:
# df_cps_09 = pd.read_csv('../datasets/cps/cpsb200910.csv')
# df_cps_11 = pd.read_csv('..datasets/cps/cpsb201110.csv')
# df_cps_12 = pd.read_csv('..datasets/cps/cpsb201210.csv')
# df_cps_13 = pd.read_csv('..datasets/cps/cpsb201310.csv')
# df_cps_14 = pd.read_csv('..datasets/cps/cpsb201410.csv')
# df_cps_15 = pd.read_csv('..datasets/cps/cpsb201510.csv')
# df_cps_16 = pd.read_csv('..datasets/cps/cpsb201610.csv')
# df_cps_17 = pd.read_csv('..datasets/cps/cpsb201710.csv')
# df_cps_18 = pd.read_csv('..datasets/cps/cpsb201810.csv')
# df_cps_19 = pd.read_csv('..datasets/cps/cpsb201910.csv')
df_cps_20b = pd.read_csv('../datasets/cps/cpsb202010.csv')


### Isolate required columns and rename

In [482]:
df_cps_20b = df_cps_20b.loc[:,['gestfips','gtco','hryear4','hefaminc','penlfact','pudis','prcitshp','peeduca']].rename(columns={'gestfips':'FIPS_state',
                             'hryear4':'Year', 'gtco':'FIPS_county','hefaminc':'HH_income',
                        'penlfact':'Disability','pudis':'Disabled','prcitshp':'Citizenship','peeduca':'Education', })
    

### Derive FIPS Code for each county

In [483]:
df_cps_20b['FIPS_county'] = np.select([df_cps_20b['FIPS_county']<10, df_cps_20b['FIPS_county']<100],
        ['00'+df_cps_20b['FIPS_county'].astype(int).astype(str), '0'+df_cps_20b['FIPS_county'].astype(int).astype(str)],
                    default= df_cps_20b['FIPS_county'].astype(int).astype(str))
df_cps_20b['FIPS_state'] = np.where(df_cps_20b['FIPS_state']<10, 
            '0'+df_cps_20b['FIPS_state'].astype(int).astype(str), df_cps_20b['FIPS_state'].astype(int).astype(str))
df_cps_20b['FIPS'] = df_cps_20b['FIPS_state'] + df_cps_20b['FIPS_county']


### Get population for each FIPS

In [484]:
pop_per_county = df_cps_20b['FIPS'].value_counts().reset_index().rename(columns={'FIPS':'Num_respondants_b','index':'FIPS'}).set_index('FIPS')

### Unstack Columns
Turn values into columns for `Citizenship` and `Education`

In [485]:
# Map Citizenship status
conditions=[ df_cps_20b['Citizenship']==5]

choices = [ 'pop_non_citizen' ]

df_cps_20b['Citizenship'] = np.select(conditions, choices,default= 'Pop_citizen')


In [486]:
# Map Education Level

conditions=[df_cps_20b['Education'] ==39,df_cps_20b['Education'] ==40,df_cps_20b['Education'] ==41,
            df_cps_20b['Education']==42, df_cps_20b['Education']==43, df_cps_20b['Education']==44,
            df_cps_20b['Education']==45, df_cps_20b['Education']==46]

choices = [ 'pop_hs_grad','pop_hs_grad', 'pop_hs_grad', 'pop_hs_grad','pop_bachelors','pop_grad_degree',
           'pop_grad_degree', 'pop_grad_degree' ]

df_cps_20b['Education'] = np.select(conditions, choices,default='Nan_degree')


### Group By County Code, and count

In [487]:
to_count = df_cps_20b.loc[:,['FIPS','Citizenship','Education']]

not_citizen = to_count[to_count['Citizenship']=='pop_non_citizen'].groupby('FIPS')[['Citizenship' ]].count().rename(columns={'Citizenship':'pop_non_citizen'})
citizen = to_count[to_count['Citizenship']=='Pop_citizen'].groupby('FIPS')[['Citizenship' ]].count().rename(columns={'Citizenship':'Pop_citizen'})
nan_degree = to_count[to_count['Education']=='Nan_degree'].groupby('FIPS')[['Education' ]].count().rename(columns={'Education':'Nan_degree'})
hs_grad = to_count[to_count['Education']=='pop_hs_grad'].groupby('FIPS')[['Education' ]].count().rename(columns={'Education':'pop_hs_grad'})
bachelors_grad = to_count[to_count['Education']=='pop_bachelors'].groupby('FIPS')[['Education' ]].count().rename(columns={'Education':'pop_bachelors'})
grad_degree = to_count[to_count['Education']=='pop_grad_degree'].groupby('FIPS')[['Education' ]].count().rename(columns={'Education':'pop_grad_degree'})


### Concat new columns together

In [488]:
# Concat together the unstacked columns with value counts for each County
df_cps_20b = pd.concat([pop_per_county,not_citizen, citizen, nan_degree,hs_grad,bachelors_grad,grad_degree], axis=1).fillna(0)

# Rename FIPS column
# df_cps_20b.rename(columns={'index':'FIPS'},inplace=True)

### Concat CPSa and CPSb Dataframes

In [489]:
df_cps_20 = pd.concat([df_cps_20a, df_cps_20b], axis=1)
df_cps_20.head()

,HH_health_insured,HH_nan_insured,HH_some_insured,HH_not_insured,HH_no_health_pub,HH_nan_insured_pub,HH_insured_pub,HH_some_insured_pub,HH_health_priv,HH_nan_insured_priv,HH_not_insured_priv,HH_some_insured_priv,HH_not_disabled,HH_nan_disabled,HH_disabled,HH_income,HH_size,Num_respondants_a,Year,Num_respondants_b,pop_non_citizen,Pop_citizen,Nan_degree,pop_hs_grad,pop_bachelors,pop_grad_degree
01000,889,490,110.0,32.0,504,490,315.0,212.0,582,490,273.0,176.0,1002,490,29.0,53980.42,1.70,1521,2020,2185,27.0,2158,903,870,261.0,151.0
01003,86,58,3.0,2.0,36,58,42.0,13.0,51,58,29.0,11.0,88,58,3.0,52303.19,1.35,149,2020,212,0.0,212,69,82,43.0,18.0
01081,49,36,7.0,2.0,29,36,16.0,13.0,31,36,15.0,12.0,57,36,1.0,53911.45,1.61,94,2020,155,7.0,148,62,57,24.0,12.0
01097,97,44,11.0,7.0,71,44,22.0,22.0,76,44,24.0,15.0,113,44,2.0,50787.05,1.72,159,2020,248,0.0,248,96,97,31.0,24.0
02000,570,374,95.0,49.0,403,374,187.0,124.0,399,374,213.0,102.0,694,374,20.0,67311.89,1.92,1088,2020,1655,31.0,1624,719,623,187.0,126.0


### Save dataframe with pickle for feature engineering

In [490]:
import pickle
with open('../pickled/cps_20_data.pickle', "wb") as output_file:
    pickle.dump(df_cps_20, output_file)

# 4. American Community Survey Data
The American Community Survey is a demographics survey program conducted by the U.S. Census Bureau. ACS 1-year estimate data is available through a Python API wrapper, called [censusdata](https://pypi.org/project/CensusData/).<br>
**Source:** United States Census Bureau [ACS Survey](https://www.census.gov/programs-surveys/acs/data.html)

In [494]:
censusdata.search('acs5', 2015, 'label', 'disab', tabletype='profile')[:50]

[('DP02PR_0070E',
  'SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO',
  'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population'),
 ('DP02PR_0070M',
  'SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO',
  'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population'),
 ('DP02PR_0070PE',
  'SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO',
  'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population'),
 ('DP02PR_0070PM',
  'SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO',
  'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population'),
 ('DP02PR_0071E',
  'SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO',
  'DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population!!With a disability'),
 ('DP02PR_0071M',
  'SELECTED

Median HH Income - DP03_0062E<br>
Avg HH Size - DP02_0015E<br>
Non citezen and foreign - DP02_0095E<br>
Population disabled - DP02_0071M<br>
population 65+ - DP02_0076E<br>
hh's with snap or food stamps - DP03_0074E<br>
hh's with no vehicle access - DP04_0058E<br>
num labor force 16+ employed - DP03_0004E<br>
Percent of fam/hh with income below poverty level - DP03_0119PE<br>
total num hh - DP02_0001E

In [492]:
def get_cps(year):
    df_cps = censusdata.download('acs5', year, censusdata.censusgeo([('county', '*')]),
                                   ['DP03_0062E', 'DP02_0071E','DP02_0061E', 'DP02_0064E', 'DP02_0065E',
                                    'DP03_0097E','DP03_0098E','DP03_0099E','DP02_0122E','DP03_0119PE', 'DP02_0015E',
                                    'DP02_0076E', 'DP04_0058E','DP02_0001E','DP02_0095E','DP03_0074E'],
                                   tabletype='profile')
    df_cps= df_cps.rename(columns={'DP03_0062E':'hh_med_income', 'DP02_0015E':'hh_avg_size', 'DP02_0095E':'pop_non_citizen',
                       'DP02_0071E': 'pop_disabled', 'DP02_0076E':'pop_65+', 'DP02_0061E':'pop_hs_grad',
                       'DP02_0064E': 'pop_bachelors', 'DP02_0065E':'pop_grad_degree','DP03_0074E':'hh_SNAP',
                       'DP02_0001E': 'num_hh', 'DP04_0058E': 'hh_no_vehicle', 
                       'DP03_0097E': 'pop_priv_health', 'DP03_0098E': 'pop_public_health', 'DP03_0099E':'pop_no_health',
                       'DP02_0122E': 'pop_total', 'DP03_0004E': 'pop_employed', 'DP03_0005E':'pop_unemployed',
                       'DP03_0119PE': 'percent_hh_poverty'}).reset_index()
    fips=[]
    for c in df_cps['index'].tolist():
        fips.append(c.geo[0][1] +c.geo[1][1])
    df_cps['FIPS'] = fips
    df_cps.drop('index',axis=1,inplace=True)
    df_cps['Year'] = str(year)
    return(df_cps)
    

In [493]:
df_cps = get_cps(2010)
for yr in [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
    df_cps = pd.concat([df_cps, get_cps(yr)])
df_cps

,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,FIPS,Year
0,38376,-8.89e+08,2376.0,260.0,330.0,-888888888,-888888888,-888888888,9642.0,14.9,2.51,-8.89e+08,684,3339.0,181.0,468,13155,2010
1,51506,-8.89e+08,13982.0,3868.0,2898.0,-888888888,-888888888,-888888888,58347.0,11.7,2.73,-8.89e+08,5386,20917.0,1779.0,2333,13157,2010
2,42081,-8.89e+08,3674.0,687.0,506.0,-888888888,-888888888,-888888888,13695.0,13.2,2.73,-8.89e+08,1392,4998.0,373.0,828,13159,2010
3,32928,-8.89e+08,4032.0,721.0,333.0,-888888888,-888888888,-888888888,14558.0,14.9,2.58,-8.89e+08,1903,5567.0,567.0,619,13161,2010
4,29268,-8.89e+08,4808.0,674.0,277.0,-888888888,-888888888,-888888888,16919.0,19.0,2.64,-8.89e+08,1963,6281.0,218.0,1040,13163,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,44717,1.42e+04,1252.0,669.0,841.0,8415,6378,1575,5.0,13.1,1788.00,1.39e+03,272,5491.0,214.0,959,47033,2019
3216,35191,4.60e+03,966.0,217.0,346.0,2327,2399,521,0.0,25.6,746.00,6.87e+02,259,2243.0,33.0,847,47095,2019
3217,57470,4.57e+05,17443.0,25676.0,69085.0,331009,143403,35777,963.0,9.5,51283.00,2.76e+04,11109,187319.0,9861.0,20359,47093,2019
3218,69023,1.96e+05,5961.0,12994.0,24206.0,132402,78477,14248,586.0,8.6,20040.00,1.42e+04,3377,72121.0,8145.0,9610,53005,2019


# 5. Demographic Data
This dataset contains columns on demographic information such as gender, race, and age, for each US county, for years 2010-2019.<br>
[Data Dictionary](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf)<br>
**Source:** United States Census Bureau [County Population Estimates](https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html)

In [178]:
# Import the dataset and inspect
df_demographics = pd.read_csv('../datasets/demographics/demographics.csv',encoding='iso-8859-1')
df_demographics.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,21295,22002,4559,5130,119,139,200,284,29,18,367,429,21633,22391,4704,5306,277,314,300,409,42,37,25875,27386,20709,21485,4512,5091,103,115,194,280,13,9,344,406,21026,21853,4647,5258,251,282,291,398,23,27,694,616,586,517,47,39,16,24,6,4,16,9,23,23,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,1411,1316,362,317,5,3,13,15,1,0,74,62,1479,1368,405,362,23,18,34,28,3,1,1778,1651,1337,1260,356,313,2,2,13,15,0,0,70,61,1402,1312,396,357,19,17,34,28,1,0,88,62,74,56,6,4,3,1,0,0,1,0,4,1,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,1521,1526,399,374,14,8,17,21,1,3,49,58,1570,1583,425,403,27,19,32,42,3,4,1933,1916,1460,1465,398,372,12,2,17,21,0,3,46,53,1506,1517,423,400,25,12,30,39,1,4,68,74,61,61,1,2,2,6,0,0,1,0,3,5,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,1658,1620,431,406,15,12,23,18,4,1,40,62,1694,1681,453,436,29,27,32,37,4,5,2105,2055,1613,1570,421,403,12,9,22,18,3,0,34,55,1643,1624,440,429,24,22,30,36,3,4,66,64,45,50,10,3,3,3,1,0,1,1,6,7,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,1628,1585,502,424,12,7,25,14,4,2,42,45,1664,1624,525,444,23,20,39,31,6,5,2153,2026,1580,1543,495,420,12,5,23,14,1,1,42,43,1616,1580,518,439,23,18,37,30,3,4,60,51,48,42,7,4,0,2,2,0,3,1,0,2,48,44,7,5,0,2,2,1,3,1


### Map categorical variables to values from data dictionary
Many features in the dataset are coded, and must be mapped to their actual values using a data dictionary using np.select statements:

In [179]:
# Isolate features to be incorporated into model
df_demographics = df_demographics.loc[:,['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP', 'TOT_POP','TOT_MALE', 'TOT_FEMALE',
    'WA_MALE','WA_FEMALE','BA_MALE','BA_FEMALE','IA_MALE','IA_FEMALE','AA_MALE','AA_FEMALE','NA_MALE','NA_FEMALE','H_MALE','H_FEMALE']]

# Map coded Year column to actual years
conditions=[((df_demographics['YEAR'] ==1) | (df_demographics['YEAR'] ==2) | (df_demographics['YEAR'] ==3)),
            df_demographics['YEAR'] ==4, df_demographics['YEAR'] ==5, df_demographics['YEAR'] ==6, 
            df_demographics['YEAR'] ==7, df_demographics['YEAR'] ==8, df_demographics['YEAR'] ==9,
            df_demographics['YEAR'] ==10, df_demographics['YEAR'] ==11, df_demographics['YEAR'] ==12]
choices = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df_demographics['YEAR'] = np.select(conditions, choices,default='N/A')

# Only retain 'All Ages' rows (where AGEGRP==0)
df_demographics = df_demographics[df_demographics['AGEGRP']==0]



### Create new columns for totals
The dataset provides data on number of males and females for each demographic, which are added together to produce a total column for each demographic:

In [180]:
df_demographics['TOT_WHITE'] = df_demographics['WA_MALE'] + df_demographics['WA_FEMALE']
df_demographics['TOT_BLACK'] = df_demographics['BA_MALE'] + df_demographics['BA_FEMALE']
df_demographics['TOT_NATIVE'] = df_demographics['IA_MALE'] + df_demographics['IA_FEMALE']
df_demographics['TOT_ASIAN'] = df_demographics['AA_MALE'] + df_demographics['AA_FEMALE']
df_demographics['TOT_PACIFIC'] = df_demographics['NA_MALE'] + df_demographics['NA_FEMALE']
df_demographics['TOT_LATINX'] = df_demographics['H_MALE'] + df_demographics['H_FEMALE']


# drop unnecessary cols
df_demographics.drop(['AGEGRP','WA_MALE','WA_FEMALE','BA_MALE','BA_FEMALE','IA_MALE','IA_FEMALE',
                      'AA_MALE','AA_FEMALE','NA_MALE','NA_FEMALE','H_FEMALE','H_MALE'], axis=1, inplace=True)

### Rename columns for interpretability

In [181]:
df_demographics.rename(columns={'YEAR':'Year','STATE':'FIPS_state', 'COUNTY':'FIPS_county', 
                                'STNAME': 'State', 'CTYNAME':'County'}, inplace=True)

### Fix format of FIPS columns
As seen in the previous datasets, FIPS state & county columns are used to create a complete FIPS column, and leading zeroes are added due to being dropped in the csv.

In [182]:
df_demographics['FIPS_county'] = np.select([df_demographics['FIPS_county']<10, df_demographics['FIPS_county']<100],
                    ['00'+df_demographics['FIPS_county'].astype(str), '0'+df_demographics['FIPS_county'].astype(str)],
                    default= df_demographics['FIPS_county'].astype(str))
df_demographics['FIPS_state'] = np.where(df_demographics['FIPS_state']<10, 
                        '0'+df_demographics['FIPS_state'].astype(str), df_demographics['FIPS_state'].astype(str))

# Create main fips code and inspect the cleaned dataset
df_demographics['FIPS'] = df_demographics['FIPS_state'] + df_demographics['FIPS_county']
df_demographics

,FIPS_state,FIPS_county,State,County,Year,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,FIPS
0,01,001,Alabama,Autauga County,2010,54571,26569,28002,43297,9689,258,484,47,1310,01001
19,01,001,Alabama,Autauga County,2010,54597,26584,28013,43313,9699,258,484,47,1310,01001
38,01,001,Alabama,Autauga County,2010,54773,26672,28101,43420,9750,251,497,46,1313,01001
57,01,001,Alabama,Autauga County,2011,55227,26981,28246,43699,9883,261,514,51,1339,01001
76,01,001,Alabama,Autauga County,2012,54954,26826,28128,43315,9949,275,552,44,1315,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716281,56,045,Wyoming,Weston County,2015,7208,3799,3409,6835,39,107,81,2,285,56045
716300,56,045,Wyoming,Weston County,2016,7220,3788,3432,6826,38,108,88,2,296,56045
716319,56,045,Wyoming,Weston County,2017,6968,3660,3308,6558,44,114,97,2,287,56045
716338,56,045,Wyoming,Weston County,2018,6924,3627,3297,6474,47,125,109,2,273,56045


# 6. Houselessness Data
This dataset contains data on houselessness rates in the US by Continuum of Care (CoC,) for the years 2009-2019.<br>
**Source:** US Dept. Housing & Urban Development (HUD) [Point in Time Estimates](https://www.hud.gov/2019-point-in-time-estimates-of-homelessness-in-US)

In [143]:
# Import the datasets
df_houseless_19 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2019')
df_houseless_18 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2018')
df_houseless_17 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2017')
df_houseless_16 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2016')
df_houseless_15 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2015')
df_houseless_14 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2014')
df_houseless_13 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2013')
df_houseless_12 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2012')
df_houseless_11 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2011')
df_houseless_10 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2010')
df_houseless_09 = pd.read_excel('../datasets/houseless/houseless_coc.xlsx', sheet_name='2009')


In [144]:
# Inspect one of the datasets
df_houseless_10.head(3)

,CoC Number,CoC Name,"Overall Homeless, 2010","Sheltered ES Homeless, 2010","Sheltered TH Homeless, 2010","Sheltered SH Homeless, 2010","Sheltered Total Homeless, 2010","Unsheltered Homeless, 2010","Homeless Individuals, 2010","Sheltered ES Homeless Individuals, 2010","Sheltered TH Homeless Individuals, 2010","Sheltered SH Homeless Individuals, 2010","Sheltered Total Homeless Individuals, 2010","Unsheltered Homeless Individuals, 2010","Homeless People in Families, 2010","Sheltered ES Homeless People in Families, 2010","Sheltered TH Homeless People in Families, 2010","Sheltered Total Homeless People in Families, 2010","Unsheltered Homeless People in Families, 2010","Homeless Family Households, 2010","Sheltered ES Homeless Family Households, 2010","Sheltered TH Homeless Family Households, 2010","Sheltered Total Homeless Family Households, 2010","Unsheltered Homeless Family Households, 2010","Sheltered SH Chronically Homeless, 2010","Chronically Homeless Individuals, 2010","Sheltered ES Chronically Homeless Individuals, 2010","Sheltered SH Chronically Homeless Individuals, 2010","Sheltered Total Chronically Homeless Individuals, 2010","Unsheltered Chronically Homeless Individuals, 2010"
0,AK-500,Anchorage CoC,1231.0,667.0,446.0,0.0,1113.0,118.0,740.0,440.0,193.0,0.0,633.0,107.0,491.0,227.0,253.0,480.0,11.0,151.0,67.0,80.0,147.0,4.0,0.0,56.0,43.0,0.0,43.0,13.0
1,AK-501,Alaska Balance of State CoC,632.0,346.0,212.0,0.0,558.0,74.0,378.0,201.0,119.0,0.0,320.0,58.0,254.0,145.0,93.0,238.0,16.0,93.0,46.0,43.0,89.0,4.0,0.0,108.0,76.0,0.0,76.0,32.0
2,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",2273.0,372.0,697.0,0.0,1069.0,1204.0,1645.0,287.0,469.0,0.0,756.0,889.0,628.0,85.0,228.0,313.0,315.0,235.0,31.0,86.0,117.0,118.0,0.0,611.0,285.0,0.0,285.0,326.0


### Trim down columns not needed for modeling

In [145]:
df_houseless_19 = df_houseless_19.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2019', 'Sheltered Total Homeless, 2019', 'Unsheltered Homeless, 2019']]
df_houseless_18 = df_houseless_18.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2018', 'Sheltered Total Homeless, 2018', 'Unsheltered Homeless, 2018']]
df_houseless_17 = df_houseless_17.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2017', 'Sheltered Total Homeless, 2017', 'Unsheltered Homeless, 2017']]
df_houseless_16 = df_houseless_16.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2016', 'Sheltered Total Homeless, 2016', 'Unsheltered Homeless, 2016']]
df_houseless_15 = df_houseless_15.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2015', 'Sheltered Total Homeless, 2015', 'Unsheltered Homeless, 2015']]
df_houseless_14 = df_houseless_14.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2014', 'Sheltered Total Homeless, 2014', 'Unsheltered Homeless, 2014']]
df_houseless_13 = df_houseless_13.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2013', 'Sheltered Total Homeless, 2013', 'Unsheltered Homeless, 2013']]
df_houseless_12 = df_houseless_12.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2012', 'Sheltered Total Homeless, 2012', 'Unsheltered Homeless, 2012']]
df_houseless_11 = df_houseless_11.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2011', 'Sheltered Total Homeless, 2011', 'Unsheltered Homeless, 2011']]
df_houseless_10 = df_houseless_10.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2010', 'Sheltered Total Homeless, 2010', 'Unsheltered Homeless, 2010']]
df_houseless_09 = df_houseless_09.loc[:,['CoC Number', 'CoC Name','Overall Homeless, 2009', 'Sheltered Total Homeless, 2009', 'Unsheltered Homeless, 2009']]



### Rename columns for interpretability

In [146]:
df_houseless_19.rename(columns={'Overall Homeless, 2019':'Tot_houseless', 
                       'Sheltered Total Homeless, 2019': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2019': 'Unsheltered_houseless'}, inplace=True)
df_houseless_18.rename(columns={'Overall Homeless, 2018':'Tot_houseless', 
                       'Sheltered Total Homeless, 2018': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2018': 'Unsheltered_houseless'}, inplace=True)
df_houseless_17.rename(columns={'Overall Homeless, 2017':'Tot_houseless', 
                       'Sheltered Total Homeless, 2017': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2017': 'Unsheltered_houseless'}, inplace=True)
df_houseless_16.rename(columns={'Overall Homeless, 2016':'Tot_houseless', 
                       'Sheltered Total Homeless, 2016': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2016': 'Unsheltered_houseless'}, inplace=True)
df_houseless_15.rename(columns={'Overall Homeless, 2015':'Tot_houseless', 
                       'Sheltered Total Homeless, 2015': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2015': 'Unsheltered_houseless'}, inplace=True)
df_houseless_14.rename(columns={'Overall Homeless, 2014':'Tot_houseless', 
                       'Sheltered Total Homeless, 2014': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2014': 'Unsheltered_houseless'}, inplace=True)
df_houseless_13.rename(columns={'Overall Homeless, 2013':'Tot_houseless', 
                       'Sheltered Total Homeless, 2013': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2013': 'Unsheltered_houseless'}, inplace=True)
df_houseless_12.rename(columns={'Overall Homeless, 2012':'Tot_houseless', 
                       'Sheltered Total Homeless, 2012': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2012': 'Unsheltered_houseless'}, inplace=True)
df_houseless_11.rename(columns={'Overall Homeless, 2011':'Tot_houseless', 
                       'Sheltered Total Homeless, 2011': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2011': 'Unsheltered_houseless'}, inplace=True)
df_houseless_10.rename(columns={'Overall Homeless, 2010':'Tot_houseless', 
                       'Sheltered Total Homeless, 2010': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2010': 'Unsheltered_houseless'}, inplace=True)
df_houseless_09.rename(columns={'Overall Homeless, 2009':'Tot_houseless', 
                       'Sheltered Total Homeless, 2009': 'Sheltered_houseless',
                       'Unsheltered Homeless, 2009': 'Unsheltered_houseless'}, inplace=True)

## Subtask: Derive houselessness at the County level using CoC's
A CoC is a regional planning body that coordinates housing and services funding for homeless families and individuals. A CoC often encompasses several counties within a State. For the sake of standardizing to all other datasets, the following process will derive a houselessness rate per CoC, and then extend apply those rates to the population of each County, to derive houselessness counts at the County level.<br>
**Method:**<br>
- Merge CoC column to demographics df to get population perCoC.<br>
- Join that with houseless df to derive houseless rate per CoC.<br>
- Join that to CoC_county to get houseless rate per county.<br>
- (optional) Join rates back with demographic df to get number of houseless, per county<br>

**Source:** [CoC-to-County Mappings](https://github.com/tomhbyrne/HUD-CoC-Geography-Crosswalk/blob/master/output/county_coc_match.csv)

### Import csv that maps counties to a CoC

In [147]:
CoC_county = pd.read_csv('../datasets/houseless/county_coc_match.csv', encoding='ISO-8859-1')
CoC_county = CoC_county.loc[:,['county_fips','coc_number']]

# rename columns
CoC_county.rename(columns={'county_fips':'FIPS'}, inplace=True)

# drop 2 rows with no FIPS
CoC_county.drop(CoC_county[CoC_county['FIPS'].isnull()].index, axis=0, inplace=True)

### Fix format of FIPS column

In [148]:
# change FIPS to string and add leading zeros if needed
CoC_county['FIPS'] = np.where(CoC_county['FIPS']<10000, 
                        '0'+CoC_county['FIPS'].astype(int).astype(str), CoC_county['FIPS'].astype(int).astype(str))

### Get population count per CoC
Data from the demographics dataset is used to get population per County. This is mapped with CoC's, to get population per CoC.

In [149]:
# get total population by county
avg = df_demographics[df_demographics['Year']=='2010'].groupby('FIPS').mean()
# merge with CoC_county to get pop count per CoC
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_10 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# repeat for each year
avg = df_demographics[df_demographics['Year']=='2011'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_11 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2012
avg = df_demographics[df_demographics['Year']=='2012'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_12 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2013
avg = df_demographics[df_demographics['Year']=='2013'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_13 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2014
avg = df_demographics[df_demographics['Year']=='2014'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_14 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2015
avg = df_demographics[df_demographics['Year']=='2015'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_15 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2016
avg = df_demographics[df_demographics['Year']=='2016'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_16 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2017
avg = df_demographics[df_demographics['Year']=='2017'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_17 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2018
avg = df_demographics[df_demographics['Year']=='2018'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_18 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()

# 2019
avg = df_demographics[df_demographics['Year']=='2019'].groupby('FIPS').mean()
merged = avg.merge(CoC_county, on='FIPS', how='left')
pop_per_coc_19 = merged.groupby('coc_number').sum()['TOT_POP'].reset_index()


### Get houselessness rate per CoC
Population count per CoC is merged with th houselessness df to derive houselessness rate per CoC

In [150]:
# Get houseless rate per CoC 
# 2010
merged = pop_per_coc_10.merge(df_houseless_10, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_10 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2011
merged = pop_per_coc_11.merge(df_houseless_11, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_11 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2012
merged = pop_per_coc_12.merge(df_houseless_12, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_12 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2013
merged = pop_per_coc_13.merge(df_houseless_13, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_13 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2014
merged = pop_per_coc_14.merge(df_houseless_14, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_14 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2015
merged = pop_per_coc_15.merge(df_houseless_15, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_15 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2016
merged = pop_per_coc_16.merge(df_houseless_16, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_16 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2017
merged = pop_per_coc_17.merge(df_houseless_17, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_17 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2018
merged = pop_per_coc_18.merge(df_houseless_18, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_18 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

# 2019
merged = pop_per_coc_19.merge(df_houseless_19, left_on='coc_number', right_on='CoC Number')
merged['Houseless_rate'] = merged['Tot_houseless']/merged['TOT_POP'] 
merged['Sheltered_rate'] = merged['Sheltered_houseless']/merged['TOT_POP'] 
merged['Unsheltered_rate'] = merged['Unsheltered_houseless']/merged['TOT_POP'] 
rates_19 = merged.loc[:,['coc_number', 'Houseless_rate','Sheltered_rate','Unsheltered_rate']]

### Houselessness rates per County
Houseless rate per CoC is mapped back to Counties, to get houseless rates per county

In [151]:
# Get houseless rate per county, for each year
df_houseless_10 = rates_10.merge(CoC_county, on='coc_number')
df_houseless_11 = rates_11.merge(CoC_county, on='coc_number')
df_houseless_12 = rates_12.merge(CoC_county, on='coc_number')
df_houseless_13 = rates_13.merge(CoC_county, on='coc_number')
df_houseless_14 = rates_14.merge(CoC_county, on='coc_number')
df_houseless_15 = rates_15.merge(CoC_county, on='coc_number')
df_houseless_16 = rates_16.merge(CoC_county, on='coc_number')
df_houseless_17 = rates_17.merge(CoC_county, on='coc_number')
df_houseless_18 = rates_18.merge(CoC_county, on='coc_number')
df_houseless_19 = rates_19.merge(CoC_county, on='coc_number')

### Add Year column to each df
Year column is added so that each df can be vertically concatenated together

In [152]:
df_houseless_10['Year'] = '2010'
df_houseless_11['Year'] = '2011'
df_houseless_12['Year'] = '2012'
df_houseless_13['Year'] = '2013'
df_houseless_14['Year'] = '2014'
df_houseless_15['Year'] = '2015'
df_houseless_16['Year'] = '2016'
df_houseless_17['Year'] = '2017'
df_houseless_18['Year'] = '2018'
df_houseless_19['Year'] = '2019'


### Concatenate all df's to create master dataframe of all years

In [153]:
df_houseless = pd.concat([df_houseless_10, df_houseless_11,df_houseless_12,df_houseless_13,df_houseless_14,df_houseless_15,
          df_houseless_16,df_houseless_17,df_houseless_18,df_houseless_19])

# Inspect cleaned houselessness df
df_houseless

,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,FIPS,Year
0,AK-500,4.00e-02,3.62e-02,3.83e-03,02020,2010
1,AK-501,1.43e-02,1.26e-02,1.68e-03,02013,2010
2,AK-501,1.43e-02,1.26e-02,1.68e-03,02016,2010
3,AK-501,1.43e-02,1.26e-02,1.68e-03,02050,2010
4,AK-501,1.43e-02,1.26e-02,1.68e-03,02060,2010
...,...,...,...,...,...,...
3158,WY-500,9.00e-03,6.94e-03,2.05e-03,56037,2019
3159,WY-500,9.00e-03,6.94e-03,2.05e-03,56039,2019
3160,WY-500,9.00e-03,6.94e-03,2.05e-03,56041,2019
3161,WY-500,9.00e-03,6.94e-03,2.05e-03,56043,2019


# 7. Rent Prices Data
This dataset contains *monthly* data from Zillow.com on 1-bedroom rent prices *by zipcode*, for the years 2014-2020.
The data is produced using the **Zillow Observed Rent Index** (ZORI,) which is a smoothed measure of the typical observed market rate rent across a given region.<br>

"ZORI is a repeat-rent index that is weighted to the rental housing stock to ensure representativeness across the entire market, not just those homes currently listed for-rent. The index is dollar-denominated by computing the mean of listed rents that fall into the 40th to 60th percentile range for all homes and apartments in a given region, which is once again weighted to reflect the rental housing stock."<br>

**Source:** [Zillow Observed Rent Index](https://www.zillow.com/research/data/)


In [154]:
# import Zillow dataset
df_rent = pd.read_csv('../datasets/rent_prices/rent_prices.csv')
df_rent.head(3)

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10
0,61639,10025,1,"New York, NY",3008.0,3020.0,3032.0,3045.0,3057.0,3069.0,3082.0,3094.0,3106.0,3118.0,3129.0,3141.0,3153.0,3162.0,3171.0,3180.0,3185.0,3190.0,3194.0,3198.0,3202.0,3206.0,3210.0,3213.0,3217.0,3219.0,3222.0,3225.0,3225.0,3226.0,3227.0,3225.0,3223.0,3221.0,3220.0,3219.0,3218.0,3219.0,3220.0,3220.0,3219.0,3217.0,3216.0,3214.0,3213.0,3211.0,3210.0,3209.0,3207.0,3207.0,3208.0,3208.0,3210.0,3211.0,3213.0,3216.0,3218.0,3221.0,3225.0,3229.0,3233.0,3239.0,3245.0,3252.0,3261.0,3271.0,3281.0,3289.0,3298.0,3306.0,3296.0,3287.0,3277.0,3249.0,3221.0,3193.0,3164.0,3134.0,3104.0,3071.0,3038.0,3005.0
1,84654,60657,2,"Chicago, IL",1577.0,1583.0,1588.0,1593.0,1598.0,1603.0,1608.0,1613.0,1618.0,1623.0,1628.0,1633.0,1638.0,1643.0,1647.0,1652.0,1656.0,1660.0,1664.0,1669.0,1673.0,1678.0,1683.0,1688.0,1693.0,1698.0,1703.0,1708.0,1712.0,1717.0,1722.0,1726.0,1731.0,1735.0,1738.0,1741.0,1744.0,1746.0,1747.0,1749.0,1751.0,1752.0,1754.0,1756.0,1757.0,1759.0,1760.0,1762.0,1763.0,1763.0,1764.0,1764.0,1764.0,1764.0,1764.0,1765.0,1766.0,1767.0,1769.0,1772.0,1774.0,1779.0,1784.0,1788.0,1795.0,1801.0,1807.0,1812.0,1817.0,1822.0,1822.0,1822.0,1822.0,1818.0,1814.0,1810.0,1806.0,1801.0,1797.0,1792.0,1786.0,1781.0
2,61637,10023,3,"New York, NY",3136.0,3145.0,3154.0,3163.0,3171.0,3179.0,3187.0,3195.0,3202.0,3210.0,3215.0,3220.0,3225.0,3229.0,3233.0,3236.0,3242.0,3248.0,3254.0,3261.0,3267.0,3274.0,3280.0,3285.0,3290.0,3290.0,3291.0,3291.0,3288.0,3284.0,3281.0,3278.0,3275.0,3272.0,3273.0,3274.0,3275.0,3276.0,3278.0,3279.0,3280.0,3281.0,3281.0,3280.0,3279.0,3279.0,3278.0,3276.0,3275.0,3277.0,3278.0,3280.0,3284.0,3288.0,3292.0,3296.0,3301.0,3305.0,3310.0,3315.0,3320.0,3327.0,3333.0,3340.0,3347.0,3355.0,3362.0,3368.0,3374.0,3380.0,3370.0,3360.0,3350.0,3324.0,3298.0,3272.0,3244.0,3217.0,3189.0,3158.0,3128.0,3097.0


### Take averages of months in each year
Data is only needed at yearly intervals, so monthly averages are calculated in a new column

In [155]:
# isolate columns corresponding to each year, and make new column
drop_14 = df_rent.columns[df_rent.columns.str.contains('2014')]
df_rent['2014'] = df_rent.loc[:,drop_14].mean(axis=1)

drop_15 = df_rent.columns[df_rent.columns.str.contains('2015')]
df_rent['2015'] = df_rent.loc[:,drop_15].mean(axis=1)

drop_16 = df_rent.columns[df_rent.columns.str.contains('2016')]
df_rent['2016'] = df_rent.loc[:,drop_16].mean(axis=1)

drop_17 = df_rent.columns[df_rent.columns.str.contains('2017')]
df_rent['2017'] = df_rent.loc[:,drop_17].mean(axis=1)

drop_18 = df_rent.columns[df_rent.columns.str.contains('2018')]
df_rent['2018'] = df_rent.loc[:,drop_18].mean(axis=1)

drop_19 = df_rent.columns[df_rent.columns.str.contains('2019')]
df_rent['2019'] = df_rent.loc[:,drop_19].mean(axis=1)

drop_20 = df_rent.columns[df_rent.columns.str.contains('2020')]
df_rent['2020'] = df_rent.loc[:,drop_20].mean(axis=1)

# drop all monthly data 
to_drop = drop_14.append(drop_15).append(drop_16).append(drop_17).append(drop_18).append(drop_19).append(drop_20)
df_rent.drop(to_drop, axis=1, inplace=True)

### Rename columns

In [156]:
df_rent.rename(columns={'RegionName':'Zipcode', 'MsaName':'City/State'}, inplace=True)

## Subtask: Map Zipcodes to Counties
Data is provided by zipcode, so it is converted to the County level using the following process:<br>
**Method:** join county data to each zipcode, then groupby county and take mean
### Import a dataset with zipcode-to-county mapping

In [157]:
zips = pd.read_csv('../datasets/rent_prices/uszips.csv')


### Clean Zipcode dataset

In [158]:
# drop some columns and rename some
zips =zips.loc[:,['zip', 'county_fips']]
zips.rename(columns={'zip': 'Zipcode', 'county_fips': 'FIPS'},inplace=True)

# add leading zero to FIPS values where needed
zips['FIPS'] = np.where(zips['FIPS']<10000, 
                        '0'+zips['FIPS'].astype(str), zips['FIPS'].astype(str))

In [159]:
# merge Zillow dataset with zipcodes dataset
merged = df_rent.merge(zips, on='Zipcode', how='left')

### Group Data by County
drop zipcode column, then group by FIPS so that there is one row per County, taking the mean of each of the zipcodes within that county

In [160]:
df_rent = merged.groupby('FIPS').mean().drop(['RegionID', 'Zipcode', 'SizeRank'],axis=1).reset_index()
df_rent

,FIPS,2014,2015,2016,2017,2018,2019,2020
0,01073,1020.00,1049.23,1070.13,1090.53,1127.70,1165.38,1199.08
1,01117,1229.76,1265.13,1282.00,1296.61,1334.00,1380.59,1416.27
2,04003,1051.25,1047.46,1035.08,1033.00,1084.79,1151.58,1217.49
3,04013,1095.67,1164.84,1224.38,1274.52,1349.59,1443.69,1537.92
4,04019,928.55,947.02,974.15,1015.56,1070.98,1135.42,1206.15
...,...,...,...,...,...,...,...,...
307,53061,1390.78,1490.70,1610.37,1717.01,1793.83,1868.80,1924.69
308,53063,808.36,844.68,900.67,959.86,1031.23,1086.46,1146.72
309,55025,1280.04,1348.67,1388.86,1418.14,1437.54,1468.61,1506.96
310,55059,689.00,695.40,741.27,782.42,818.42,872.10,921.80


### Break down into separate df's by year, and add year column
Dataset must be reformatted to follow the same pattern as all others, so that they can ultimately be concatenated

In [161]:
# 2014
# isolate relevant columns, rename, and creat year column
df_rent_14 = df_rent[['FIPS', '2014']]
df_rent_14.rename(columns={'2014':'Rent'}, inplace=True)
df_rent_14['Year'] = '2014'

# 2015
df_rent_15 = df_rent[['FIPS', '2015']]
df_rent_15.rename(columns={'2015':'Rent'}, inplace=True)
df_rent_15['Year'] = '2015'

# 2016
df_rent_16 = df_rent[['FIPS', '2016']]
df_rent_16.rename(columns={'2016':'Rent'}, inplace=True)
df_rent_16['Year'] = '2016'


# 2017
df_rent_17 = df_rent[['FIPS', '2017']]
df_rent_17.rename(columns={'2017':'Rent'}, inplace=True)
df_rent_17['Year'] = '2017'

# 2018
df_rent_18 = df_rent[['FIPS', '2018']]
df_rent_18.rename(columns={'2018':'Rent'}, inplace=True)
df_rent_18['Year'] = '2018'

# 2019
df_rent_19 = df_rent[['FIPS', '2019']]
df_rent_19.rename(columns={'2019':'Rent'}, inplace=True)
df_rent_19['Year'] = '2019'

# 2020
df_rent_20 = df_rent[['FIPS', '2020']]
df_rent_20.rename(columns={'2020':'Rent'}, inplace=True)
df_rent_20['Year'] = '2020'


### Concatenate all df's to create master dataframe of all years

In [162]:
df_rent = pd.concat([df_rent_14,df_rent_15,df_rent_16,df_rent_17,df_rent_18,df_rent_19,df_rent_20])
df_rent

,FIPS,Rent,Year
0,01073,1020.00,2014
1,01117,1229.76,2014
2,04003,1051.25,2014
3,04013,1095.67,2014
4,04019,928.55,2014
...,...,...,...
307,53061,1924.69,2020
308,53063,1146.72,2020
309,55025,1506.96,2020
310,55059,921.80,2020


# 8. Businesses Data
This dataset contains data on all businesses in the US at the County level, for years 2009-2018. The dataset is used below to get Food Retail data, which includes grocery stores, wholesalers, and restaraunts.<br>
[Data Dictionary](https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/2018_record_layouts/county-layout-2018.txt)<br>
[Industry (naics) Conversion](https://www2.census.gov/programs-surveys/cbp/technical-documentation/reference/naics-descriptions/naics2017.txt)<br>

**Source:** US Census Bureau [County Business Patterns](https://www.census.gov/programs-surveys/cbp/data/datasets.html)

In [100]:
df_business_09 = pd.read_csv('../datasets/businesses/bus_09.txt')
df_business_10 = pd.read_csv('../datasets/businesses/bus_10.txt')
df_business_11 = pd.read_csv('../datasets/businesses/bus_11.txt')
df_business_12 = pd.read_csv('../datasets/businesses/bus_12.txt')
df_business_13 = pd.read_csv('../datasets/businesses/bus_13.txt')
df_business_14 = pd.read_csv('../datasets/businesses/bus_14.txt')
df_business_15 = pd.read_csv('../datasets/businesses/bus_15.txt')
df_business_16 = pd.read_csv('../datasets/businesses/bus_16.txt')
df_business_17 = pd.read_csv('../datasets/businesses/bus_17.txt')
df_business_18 = pd.read_csv('../datasets/businesses/bus_18.txt')

# inspect features
df_business_10.head(3)

,fipstate,fipscty,naics,empflag,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,censtate,cencty
0,1,1,------,NaN,G,10167,G,63783,G,273052,871,447,181,128,72,32,9,1,1,0,0,0,0,0,63,1
1,1,1,11----,NaN,H,33,H,226,H,1097,6,3,2,1,0,0,0,0,0,0,0,0,0,0,63,1
2,1,1,113///,NaN,G,27,G,188,G,945,5,3,1,1,0,0,0,0,0,0,0,0,0,0,63,1


### Drop unneeded columns and rename for interpretability

In [101]:
# 2009
df_business_09 = df_business_09.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_09 = df_business_09.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2010
df_business_10 = df_business_10.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_10 = df_business_10.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2011
df_business_11 = df_business_11.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_11 = df_business_11.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2012
df_business_12 = df_business_12.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_12 = df_business_12.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2013
df_business_13 = df_business_13.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_13 = df_business_13.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2014
df_business_14 = df_business_14.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_14 = df_business_14.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2015
df_business_15 = df_business_15.loc[:,['FIPSTATE', 'FIPSCTY', 'NAICS', 'EST']]
df_business_15 = df_business_15.rename(columns={'FIPSTATE': 'FIPS_state', 'FIPSCTY':'FIPS_county', 
                                'NAICS':'Industry','EST':'Num_establishments'})
# 2016
df_business_16 = df_business_16.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_16 = df_business_16.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2017
df_business_17 = df_business_17.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_17 = df_business_17.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})
# 2018
df_business_18 = df_business_18.loc[:,['fipstate', 'fipscty', 'naics', 'est']]
df_business_18 = df_business_18.rename(columns={'fipstate': 'FIPS_state', 'fipscty':'FIPS_county', 
                                'naics':'Industry','est':'Num_establishments'})


### Reformat FIPS codes and join together state+county codes

In [102]:
# Add leading zeros to FIPS codes, and concat state and county FIPS codes
# 2009
df_business_09['FIPS_county'] = np.select([df_business_09['FIPS_county']<10, df_business_09['FIPS_county']<100],
                    ['00'+df_business_09['FIPS_county'].astype(str), '0'+df_business_09['FIPS_county'].astype(str)],
                    default= df_business_09['FIPS_county'].astype(str))
df_business_09['FIPS_state'] = np.where(df_business_09['FIPS_state']<10, 
                        '0'+df_business_09['FIPS_state'].astype(str), df_business_09['FIPS_state'].astype(str))
df_business_09['FIPS'] = df_business_09['FIPS_state'] + df_business_09['FIPS_county']

# 2010
df_business_10['FIPS_county'] = np.select([df_business_10['FIPS_county']<10, df_business_10['FIPS_county']<100],
                    ['00'+df_business_10['FIPS_county'].astype(str), '0'+df_business_10['FIPS_county'].astype(str)],
                    default= df_business_10['FIPS_county'].astype(str))
df_business_10['FIPS_state'] = np.where(df_business_10['FIPS_state']<10, 
                        '0'+df_business_10['FIPS_state'].astype(str), df_business_10['FIPS_state'].astype(str))
df_business_10['FIPS'] = df_business_10['FIPS_state'] + df_business_10['FIPS_county']

# 2011
df_business_11['FIPS_county'] = np.select([df_business_11['FIPS_county']<10, df_business_11['FIPS_county']<100],
                    ['00'+df_business_11['FIPS_county'].astype(str), '0'+df_business_11['FIPS_county'].astype(str)],
                    default= df_business_11['FIPS_county'].astype(str))
df_business_11['FIPS_state'] = np.where(df_business_11['FIPS_state']<10, 
                        '0'+df_business_11['FIPS_state'].astype(str), df_business_11['FIPS_state'].astype(str))
df_business_11['FIPS'] = df_business_11['FIPS_state'] + df_business_11['FIPS_county']

# 2012
df_business_12['FIPS_county'] = np.select([df_business_12['FIPS_county']<10, df_business_12['FIPS_county']<100],
                    ['00'+df_business_12['FIPS_county'].astype(str), '0'+df_business_12['FIPS_county'].astype(str)],
                    default= df_business_12['FIPS_county'].astype(str))
df_business_12['FIPS_state'] = np.where(df_business_12['FIPS_state']<10, 
                        '0'+df_business_12['FIPS_state'].astype(str), df_business_12['FIPS_state'].astype(str))
df_business_12['FIPS'] = df_business_12['FIPS_state'] + df_business_12['FIPS_county']

# 2013
df_business_13['FIPS_county'] = np.select([df_business_13['FIPS_county']<10, df_business_13['FIPS_county']<100],
                    ['00'+df_business_13['FIPS_county'].astype(str), '0'+df_business_13['FIPS_county'].astype(str)],
                    default= df_business_13['FIPS_county'].astype(str))
df_business_13['FIPS_state'] = np.where(df_business_13['FIPS_state']<10, 
                        '0'+df_business_13['FIPS_state'].astype(str), df_business_13['FIPS_state'].astype(str))
df_business_13['FIPS'] = df_business_13['FIPS_state'] + df_business_13['FIPS_county']

# 2014
df_business_14['FIPS_county'] = np.select([df_business_14['FIPS_county']<10, df_business_14['FIPS_county']<100],
                    ['00'+df_business_14['FIPS_county'].astype(str), '0'+df_business_14['FIPS_county'].astype(str)],
                    default= df_business_14['FIPS_county'].astype(str))
df_business_14['FIPS_state'] = np.where(df_business_14['FIPS_state']<10, 
                        '0'+df_business_14['FIPS_state'].astype(str), df_business_14['FIPS_state'].astype(str))
df_business_14['FIPS'] = df_business_14['FIPS_state'] + df_business_14['FIPS_county']

# 2015
df_business_15['FIPS_county'] = np.select([df_business_15['FIPS_county']<10, df_business_15['FIPS_county']<100],
                    ['00'+df_business_15['FIPS_county'].astype(str), '0'+df_business_15['FIPS_county'].astype(str)],
                    default= df_business_15['FIPS_county'].astype(str))
df_business_15['FIPS_state'] = np.where(df_business_15['FIPS_state']<10, 
                        '0'+df_business_15['FIPS_state'].astype(str), df_business_15['FIPS_state'].astype(str))
df_business_15['FIPS'] = df_business_15['FIPS_state'] + df_business_15['FIPS_county']

# 2016
df_business_16['FIPS_county'] = np.select([df_business_16['FIPS_county']<10, df_business_16['FIPS_county']<100],
                    ['00'+df_business_16['FIPS_county'].astype(str), '0'+df_business_16['FIPS_county'].astype(str)],
                    default= df_business_16['FIPS_county'].astype(str))
df_business_16['FIPS_state'] = np.where(df_business_16['FIPS_state']<10, 
                        '0'+df_business_16['FIPS_state'].astype(str), df_business_16['FIPS_state'].astype(str))
df_business_16['FIPS'] = df_business_16['FIPS_state'] + df_business_16['FIPS_county']

# 2017
df_business_17['FIPS_county'] = np.select([df_business_17['FIPS_county']<10, df_business_17['FIPS_county']<100],
                    ['00'+df_business_17['FIPS_county'].astype(str), '0'+df_business_17['FIPS_county'].astype(str)],
                    default= df_business_17['FIPS_county'].astype(str))
df_business_17['FIPS_state'] = np.where(df_business_17['FIPS_state']<10, 
                        '0'+df_business_17['FIPS_state'].astype(str), df_business_17['FIPS_state'].astype(str))
df_business_17['FIPS'] = df_business_17['FIPS_state'] + df_business_17['FIPS_county']

# 2018
df_business_18['FIPS_county'] = np.select([df_business_18['FIPS_county']<10, df_business_18['FIPS_county']<100],
                    ['00'+df_business_18['FIPS_county'].astype(str), '0'+df_business_18['FIPS_county'].astype(str)],
                    default= df_business_18['FIPS_county'].astype(str))
df_business_18['FIPS_state'] = np.where(df_business_18['FIPS_state']<10, 
                        '0'+df_business_18['FIPS_state'].astype(str), df_business_18['FIPS_state'].astype(str))
df_business_18['FIPS'] = df_business_18['FIPS_state'] + df_business_18['FIPS_county']



### Isolate rows for each type of food establishment
Use the NAICS data dictionary to map industry codes to each type of food establishment. Food retailers are broken into three categories: Restaraunts, Wholesalers, and Grocery stores.

In [103]:
# 2009
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_09[df_business_09['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_09[df_business_09['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_09[df_business_09['Industry'].isin(wholesale)].index

# 2010
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_10[df_business_10['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_10[df_business_10['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_10[df_business_10['Industry'].isin(wholesale)].index

# 2011
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_11[df_business_11['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_11[df_business_11['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_11[df_business_11['Industry'].isin(wholesale)].index

# 2012
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_12[df_business_12['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_12[df_business_12['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_12[df_business_12['Industry'].isin(wholesale)].index

# 2013
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_13[df_business_13['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_13[df_business_13['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_13[df_business_13['Industry'].isin(wholesale)].index

# 2014
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_14[df_business_14['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_14[df_business_14['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_14[df_business_14['Industry'].isin(wholesale)].index

# 2015
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_15[df_business_15['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_15[df_business_15['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_15[df_business_15['Industry'].isin(wholesale)].index

# 2016
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_16[df_business_16['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_16[df_business_16['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_16[df_business_16['Industry'].isin(wholesale)].index

# 2017
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_17[df_business_17['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_17[df_business_17['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_17[df_business_17['Industry'].isin(wholesale)].index

# 2018
restaraunts = ['72233/', '722330', '722///', '7224//', '72241/', '722410', '7225//', '72251/', '722511', '722513', '722514', '722515']
restaraunt_indices = df_business_18[df_business_18['Industry'].isin(restaraunts)].index
grocery = ['445///', '4451//', '44511/', '445110', '44512/', '445120', '4452//', '44521/', '445210', '44522/', '445220', '44523/', '445230', '44529/', '445291', '445292', '445299', '4453//', '44531/', '445310', '446///', '4461//', '44611/', '446110']
grocery_indices = df_business_18[df_business_18['Industry'].isin(grocery)].index
wholesale = ['4244//', '42441/', '424410', '42442/', '424420', '42443/', '424430', '42444/', '424440', '42445/', '424450', '42446/', '424460', '42447/', '424470', '42448/', '424480', '42449/', '424490', '4245//', '42451/', '424510', '42452/', '424520', '42459/', '424590']
wholesale_indices = df_business_18[df_business_18['Industry'].isin(wholesale)].index


### Derive # of businesses in each county
Group dataframe by type of establishment, and sum # of businesses in each county

In [104]:
# 2009
restaraunt_df = df_business_09.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_09.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_09.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_09 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2010
restaraunt_df = df_business_10.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_10.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_10.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_10 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2011
restaraunt_df = df_business_11.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_11.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_11.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_11 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2012
restaraunt_df = df_business_12.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_12.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_12.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_12 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2013
restaraunt_df = df_business_13.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_13.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_13.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_13 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2014
restaraunt_df = df_business_14.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_14.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_14.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_14 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2015
restaraunt_df = df_business_15.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_15.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_15.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_15 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2016
restaraunt_df = df_business_16.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_16.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_16.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_16 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2017
restaraunt_df = df_business_17.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_17.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_17.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_17 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')

# 2018
restaraunt_df = df_business_18.loc[restaraunt_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
restaraunt_df.rename(columns={'Num_establishments':'Num_restaraunts'},inplace=True)
grocery_df = df_business_18.loc[grocery_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
grocery_df.rename(columns={'Num_establishments':'Num_grocery'},inplace=True)
wholesale_df = df_business_18.loc[wholesale_indices,['FIPS', 'Num_establishments']].groupby('FIPS').sum().reset_index()
wholesale_df.rename(columns={'Num_establishments':'Num_wholesale'},inplace=True)
# merge three df's together
df_food_18 = wholesale_df.merge(restaraunt_df, how='outer',on='FIPS').merge(grocery_df, on='FIPS', how='outer')



### Add Year column to each df and impute nulls with zero

In [105]:
df_food_09['Year'] = '2009'
df_food_09.fillna(0, inplace=True)
df_food_10['Year'] = '2010'
df_food_10.fillna(0, inplace=True)
df_food_11['Year'] = '2011'
df_food_11.fillna(0, inplace=True)
df_food_12['Year'] = '2012'
df_food_12.fillna(0, inplace=True)
df_food_13['Year'] = '2013'
df_food_13.fillna(0, inplace=True)
df_food_14['Year'] = '2014'
df_food_14.fillna(0, inplace=True)
df_food_15['Year'] = '2015'
df_food_15.fillna(0, inplace=True)
df_food_16['Year'] = '2016'
df_food_16.fillna(0, inplace=True)
df_food_17['Year'] = '2017'
df_food_17.fillna(0, inplace=True)
df_food_18['Year'] = '2018'
df_food_18.fillna(0, inplace=True)


### Concatenate all df's to create master dataframe of all years

In [106]:
df_food = pd.concat([df_food_09,df_food_10,df_food_11,df_food_12,df_food_13,df_food_14,df_food_15,
          df_food_16,df_food_17,df_food_18])
df_food

,FIPS,Num_wholesale,Num_restaraunts,Num_grocery,Year
0,01001,71.0,13.0,109.0,2009
1,01005,5.0,7.0,109.0,2009
2,01007,3.0,21.0,92.0,2009
3,01009,19.0,32.0,186.0,2009
4,01013,13.0,106.0,29.0,2009
...,...,...,...,...,...
3105,29211,0.0,0.0,18.0,2018
3106,46121,0.0,0.0,12.0,2018
3107,48155,0.0,0.0,3.0,2018
3108,51045,0.0,0.0,14.0,2018


# Merge together all dataframes
After each of the seven datasets (df_rent,df_houseless, df_cps, df_demographics, df_unemployment, df_FA, and df_food) has been cleaned and uniformly formatted, they are merged together to create a main dataframe with all of the features: 

In [505]:
# merge rent and houseless dataframes together, using FIPS and Year
merge1 = df_rent.merge(df_houseless, on=['FIPS','Year'],how='outer')


In [506]:
# merge demographics dataframe with previous merge, using FIPS and Year
merge2 = merge1.merge(df_cps, on=['FIPS','Year'],how='outer')


In [507]:
merge3 = merge2.merge(df_demographics, on=['FIPS','Year'],how='outer')

In [508]:
# merge unemployment dataframe with previous merge, using FIPS and Year
merge4 = merge3.merge(df_unemployment, on=['FIPS','Year','FIPS_state','FIPS_county'],how='outer')


In [509]:
# merge food insecurity dataframe with previous merge, using FIPS and Year
merge5 = merge4.merge(df_FA, on=['FIPS','Year'],how='outer')


In [510]:
# merge food retail dataframe with previous merge, using FIPS and Year
df_all = merge5.merge(df_food, on=['FIPS','Year'],how='outer')

### Inspect the final dataframe

In [511]:
df_all

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,FIPS_state,FIPS_county,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,State/County,Total_workforce,Employed,Unemployed,Unemployment_rate,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery
0,01073,1020.00,2014,AL-500,0.01,9.13e-03,4.13e-03,45239.0,94584.0,117854.0,81626.0,52774.0,4.32e+05,2.12e+05,81336.0,6.59e+05,14.8,2.48,87036.0,93630.0,2.59e+05,17519.0,39967.0,01,073,Alabama,Jefferson County,6.60e+05,3.12e+05,3.47e+05,3.56e+05,284082.0,2029.0,10378.0,347.0,2.41e+04,"Jefferson County, AL",3.1e+05,2.9e+05,2e+04,6.3,0.20,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0
1,01117,1229.76,2014,AL-500,0.01,9.13e-03,4.13e-03,69723.0,22792.0,28911.0,35773.0,18511.0,1.60e+05,4.24e+04,19175.0,2.01e+05,6.2,2.65,23404.0,19762.0,7.48e+04,7624.0,4706.0,01,117,Alabama,Shelby County,2.06e+05,1.00e+05,1.06e+05,1.74e+05,24247.0,805.0,4403.0,101.0,1.19e+04,"Shelby County, AL",1.1e+05,1e+05,4.8e+03,4.5,0.10,SNAP,Other Nutrition Program,3.37,1.0,743.0,2706.0
2,04003,1051.25,2014,AZ-500,0.01,7.85e-03,6.10e-03,45974.0,20341.0,21109.0,12968.0,7566.0,7.61e+04,5.05e+04,14868.0,1.31e+05,13.1,2.47,23593.0,16328.0,4.88e+04,7947.0,7812.0,04,003,Arizona,Cochise County,1.27e+05,6.47e+04,6.27e+04,1.12e+05,5737.0,2165.0,2757.0,510.0,4.44e+04,"Cochise County, AZ",5.1e+04,4.7e+04,4.3e+03,8.4,0.16,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.81,800.0,72.0,340.0
3,04013,1095.67,2014,AZ-502,0.01,1.03e-02,2.23e-03,53689.0,399455.0,593094.0,490927.0,273108.0,2.44e+06,1.20e+06,646167.0,3.95e+06,12.7,2.74,507428.0,546028.0,1.42e+06,373532.0,171581.0,04,013,Arizona,Maricopa County,4.09e+06,2.02e+06,2.07e+06,3.45e+06,235660.0,112383.0,172425.0,11190.0,1.24e+06,"Maricopa County, AZ",2e+06,1.8e+06,1.1e+05,5.8,0.16,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.90,2389.0,16857.0,6320.0
4,04019,928.55,2014,AZ-501,0.02,1.62e-02,4.42e-03,46233.0,133694.0,149147.0,115392.0,81406.0,5.92e+05,3.65e+05,141211.0,9.93e+05,13.2,2.50,162075.0,149710.0,3.86e+05,69636.0,57099.0,04,019,Arizona,Pima County,1.00e+06,4.95e+05,5.10e+05,8.58e+05,41043.0,42683.0,31905.0,2266.0,3.63e+05,"Pima County, AZ",4.6e+05,4.4e+05,2.8e+04,6,0.15,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.85,3591.0,838.0,1604.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85989,49999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,0.0
85990,51999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0.0,0.0
85991,53999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0.0,0.0
85992,54999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0


### Save the cleaned dataset using Pickle

In [512]:

with open('../pickled/partially_cleaned_data.pickle', "wb") as output_file:
    pickle.dump(df_all, output_file)